In [ ]:
import os
import time
import shutil
import pandas as pd
from openpyxl import load_workbook
import win32com.client as win32

# File Paths (Update these)
input_file = r"C:\Path\To\Reportnames.xlsx"
template_file = r"C:\Path\To\TemplateWorkbook.xlsx"
output_folder = r"C:\Path\To\OutputFolder"

# Ensure output folder exists
os.makedirs(output_folder, exist_ok=True)

# Read input data
df = pd.read_excel(input_file, usecols=["RTN CODE CHILD", "Report Name"])

# Start Excel application (COM for Refreshing)
excel = win32.Dispatch("Excel.Application")
excel.Visible = False  # Run in background

# Loop through each row in input file
for index, row in df.iterrows():
    RTN = row["RTN CODE CHILD"]
    ReportName = row["Report Name"]

    if pd.isna(RTN) or pd.isna(ReportName):  # Skip empty rows
        continue

    # Open template workbook
    wb = excel.Workbooks.Open(template_file)
    ws = wb.Sheets("Setup")

    # Paste values
    ws.Range("D3").Value = RTN
    ws.Range("D4").Value = ReportName

    # Refresh data
    wb.RefreshAll()
    time.sleep(5)  # Wait for refresh

    # Save a temporary copy (Retains formatting)
    temp_copy_path = os.path.join(output_folder, "Temp_Copy.xlsx")
    wb.SaveCopyAs(temp_copy_path)
    wb.Close(SaveChanges=False)

    # Open the copied workbook in Excel (COM method ensures proper formatting)
    new_wb = excel.Workbooks.Open(temp_copy_path)

    # Select all sheets and convert formulas to values using "Paste Special"
    for sheet in new_wb.Sheets:
        sheet.Cells.Copy()
        sheet.Cells.PasteSpecial(Paste=win32.constants.xlPasteValues)

    # Save the final file
    final_name = os.path.join(output_folder, f"{ReportName}.xlsx")
    new_wb.SaveAs(final_name, FileFormat=51)  # 51 = xlsx format
    new_wb.Close(SaveChanges=True)

# Quit Excel
excel.Quit()

print("All files processed successfully!")


In [ ]:
import pandas as pd
import os

# Define input/output paths
input_folder = "Input"
output_file = "JML_Report.xlsx"
input_file = os.path.join(input_folder, "your_excel_file.xlsx")  # Update your filename

# Load Excel file
df = pd.read_excel(input_file, sheet_name="Headcount Employee Detail", dtype=str)

# Convert required columns to string for lookups
df["Employee ID"] = df["Employee ID"].astype(str)
df["Functional Manager Employee ID"] = df["Functional Manager Employee ID"].astype(str)
df["BF Level 2 Name"] = df["BF Level 2 Name"].astype(str)

# Create a lookup dictionary for quick access
lookup_dict = df.set_index("Employee ID")[["Functional Manager Employee ID", "BF Level 2 Name"]].to_dict("index")

# Step 1: Filter for 'Central Managed Services' employees
df_cms = df[df["BF Level 2 Name"] == "Central Managed Services"].copy()

# Initialize columns for tracking hierarchy
for i in range(1, 6):  # Up to 5 levels
    df_cms[f"Manager{i}"] = ""
    df_cms[f"Check{i}"] = ""

# Function to recursively fetch Functional Manager and BF Level
def track_manager_hierarchy(emp_id, max_level=5):
    managers = []
    bf_levels = []
    
    for i in range(1, max_level + 1):
        manager_id = lookup_dict.get(emp_id, {}).get("Functional Manager Employee ID")
        bf_level = lookup_dict.get(emp_id, {}).get("BF Level 2 Name")

        if not manager_id or pd.isna(manager_id):
            break  # Stop if no manager exists
        
        managers.append(manager_id)
        bf_levels.append(bf_level)

        # Move to the next level (next manager)
        emp_id = manager_id  

    return managers, bf_levels

# Step 2: Iterate over each employee and fetch hierarchy details
for index, row in df_cms.iterrows():
    emp_id = row["Employee ID"]
    managers, bf_levels = track_manager_hierarchy(emp_id, max_level=5)

    for i, (manager, bf_level) in enumerate(zip(managers, bf_levels), start=1):
        df_cms.at[index, f"Manager{i}"] = manager
        df_cms.at[index, f"Check{i}"] = bf_level

# Step 3: Identify rows where any "Check" column has 'Finance'
check_cols = [f"Check{i}" for i in range(1, 6)]
df_cms["Updated BF Level2"] = df_cms[check_cols].apply(lambda x: "CMS Finance" if "Finance" in x.values else "", axis=1)

# Step 4: Save full df_cms (with all original + Check/Manager columns) in "CMS_Details"
with pd.ExcelWriter(output_file, engine="xlsxwriter") as writer:
    df_cms.to_excel(writer, sheet_name="CMS_Details", index=False)  # Save full CMS details for reference

    # Step 5: Filter rows where 'Updated BF Level2' is 'CMS Finance'
    cms_df = df_cms[df_cms["Updated BF Level2"] == "CMS Finance"].copy()

    # Step 6: Read original sheet again for Finance rows
    df_finance = df[df["BF Level 2 Name"] == "Finance"].copy()
    df_finance["Updated BF Level2"] = "Finance"

    # Step 7: Remove extra columns before appending to `df_finance`
    cms_df_cleaned = cms_df[df_finance.columns]

    # Step 8: Reset index types for proper concatenation
    cms_df_cleaned = cms_df_cleaned.reset_index(drop=True)
    df_finance = df_finance.reset_index(drop=True)

    # Step 9: Concatenate cleaned `cms_df` with `df_finance`
    final_df = pd.concat([df_finance, cms_df_cleaned], ignore_index=True)

    # Step 10: Save the final `JML Report`
    final_df.to_excel(writer, sheet_name="JML Report", index=False)

print("✅ Process completed successfully! The final file is saved as:", output_file)


In [ ]:
import pandas as pd

# Load Files
output_file = pd.read_excel('Output.xlsx', sheet_name='Sheet1')
input_file = pd.read_excel('Input.xlsx', sheet_name='Sheet1')
mapping_file = pd.ExcelFile('Mapping.xlsx')
ospd_file = pd.read_excel('OSPD.xlsb', sheet_name='CC-List')

# Load Mapping Sheets
gr_mapping = mapping_file.parse('GR')
anaplan_mapping = mapping_file.parse('Anaplan')
country_mapping = mapping_file.parse('Country Mapping')

# Step 2: Add 'Classification' to Output File
output_file = output_file.merge(gr_mapping[['Role Type', 'Classification']], on='Role Type', how='left')

# Step 3: Add 'Classification' to Input File
input_file = input_file.merge(anaplan_mapping[['Position class', 'Classification']], left_on='Role Type', right_on='Position class', how='left')

# Step 4: Add 'Country' to Input File based on 'Country R3'
input_file = input_file.merge(country_mapping[['Country R3', 'Country']], on='Country R3', how='left')

# Step 5: Add 'RTN Code' from OSPD file
input_file = input_file.merge(ospd_file[['L4_BF_Description', 'L4_BF_ID']], left_on='Business Framework', right_on='L4_BF_Description', how='left')
input_file.rename(columns={'L4_BF_ID': 'RTN Code'}, inplace=True)

# Step 6: Create Unique Key for Input File
input_file['Unique Key'] = input_file['Classification'] + ' ' + input_file['RTN Code'] + ' ' + input_file['Country']

# Step 7: Create Unique Key for Output File
output_file['Unique Key'] = output_file['Classification'] + ' ' + output_file['RTN Code L4'] + ' ' + output_file['Country']

# Step 8: Create Secondary Keys
input_file['Secondary Key'] = input_file['RTN Code'] + ' ' + input_file['Country']
output_file['Secondary Key'] = output_file['RTN Code L4'] + ' ' + output_file['Country']

# Step 9: Lookup Data from Input to Output File
lookup_cols = ['Business Framework', 'Country', 'Classification']
out_cols = ['A_L4', 'A_Country', 'A_Role Type']
output_file[out_cols] = output_file[['Unique Key']].merge(input_file[['Unique Key'] + lookup_cols], on='Unique Key', how='left')[lookup_cols]

# Step 10: Aggregate Cost & FTE based on Unique and Secondary Keys
agg_data = input_file.groupby('Unique Key').agg({'Cost': 'sum', 'FTE': 'sum'}).reset_index()
output_file = output_file.merge(agg_data, on='Unique Key', how='left').rename(columns={'Cost': 'A_DC', 'FTE': 'A_FTE'})

# Aggregate for Secondary Key
agg_data_sec = input_file.groupby('Secondary Key').agg({'Cost': 'sum', 'FTE': 'sum'}).reset_index()
output_file = output_file.merge(agg_data_sec, on='Secondary Key', how='left').rename(columns={'Cost': 'Total_DC', 'FTE': 'Total_FTE'})

# Save the final processed Output File
output_file.to_excel('Processed_Output.xlsx', index=False)

print("Processing completed. Output saved as 'Processed_Output.xlsx'")


In [ ]:
import pandas as pd
import os

# Define input and output file paths
input_folder = "Input"
input_file = os.path.join(input_folder, "your_excel_file.xlsx")  # Replace with actual filename
output_file = "JML_Report.xlsx"

# Read the Excel file
df = pd.read_excel(input_file, sheet_name="Headcount Employee Detail", dtype=str)  # Read everything as text

# Filter only "Central Managed Services"
df_cms = df[df["BF Level 2 Name"] == "Central Managed Services"].copy()

# Initialize columns for tracking manager levels
manager_cols = ["Check", "Manager", "Check2", "Manager2", "Check3", "Manager3", "Check4", "Manager4", "Check5", "Manager5", "Check6"]

# Add empty columns
for col in manager_cols:
    df_cms[col] = ""

# Create a dictionary for fast lookups (Employee ID → [Manager ID, BF Level])
lookup_dict = df.set_index("Employee ID")[["Functional Manager Employee ID", "BF Level 2 Name"]].to_dict(orient="index")

# Function to track hierarchy up to 5 levels
def track_manager_hierarchy(emp_id):
    manager_data = {"Check": "", "Manager": "", "Check2": "", "Manager2": "", "Check3": "", "Manager3": "", "Check4": "", "Manager4": "", "Check5": "", "Manager5": "", "Check6": ""}
    current_emp = emp_id

    for i in range(1, 7):  # Loop for levels 1 to 6
        if current_emp in lookup_dict:
            manager_id = lookup_dict[current_emp]["Functional Manager Employee ID"]
            bf_level = lookup_dict[current_emp]["BF Level 2 Name"]

            # Save in corresponding columns
            if i == 1:
                manager_data["Check"] = bf_level
                manager_data["Manager"] = manager_id
            else:
                manager_data[f"Check{i}"] = bf_level
                manager_data[f"Manager{i}"] = manager_id
            
            # Move up the hierarchy
            current_emp = manager_id
        else:
            break  # Stop if no manager found

    return pd.Series(manager_data)

# Apply the function to populate manager levels
df_cms[manager_cols] = df_cms["Employee ID"].apply(track_manager_hierarchy)

# Identify rows where any "Check" column has 'Finance'
df_cms["Updated BF Level2"] = df_cms[["Check", "Check2", "Check3", "Check4", "Check5", "Check6"]].apply(lambda x: "CMS Finance" if "Finance" in x.values else "", axis=1)

# Keep only relevant rows
cms_df = df_cms[df_cms["Updated BF Level2"] == "CMS Finance"]

# Read Finance rows from original file
df_finance = df[df["BF Level 2 Name"] == "Finance"].copy()
df_finance["Updated BF Level2"] = "Finance"

# Append cms_df rows to finance rows
final_df = pd.concat([df_finance, cms_df], ignore_index=True)

# Save to a new Excel file with sheet name "JML Report"
with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
    final_df.to_excel(writer, sheet_name="JML Report", index=False)

print("✅ JML Report generated successfully!")


In [ ]:
import pandas as pd
from openpyxl.styles import PatternFill
from openpyxl import load_workbook
from datetime import datetime

# Define the yellow fill for highlighting
yellow_fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")

# Load your Excel file
file_path = "your_file.xlsx"
df = pd.read_excel(file_path)
wb = load_workbook(file_path)
ws = wb.active

# Ensure all expected columns are present
required_columns = ['Original Date', 'Date', 'Type', 'Description']  # Add more columns as needed
for col in required_columns:
    if col not in df.columns:
        df[col] = ""  # Add missing columns with blank values

# Function to safely convert date values
def safe_convert_date(date_value):
    if pd.isnull(date_value):  # Handle missing values
        return None
    if isinstance(date_value, datetime):  # Already a datetime object
        return date_value
    if isinstance(date_value, str):  # Try parsing strings
        for fmt in ("%d/%m/%Y", "%Y-%m-%d", "%m/%d/%Y"):  # Common date formats
            try:
                return datetime.strptime(date_value, fmt)
            except ValueError:
                continue
    return None  # Return None if conversion fails

# Ensure Date and Original Date are in proper datetime format
if 'Date' in df.columns:
    df['Date'] = df['Date'].apply(safe_convert_date)
if 'Original Date' in df.columns:
    df['Original Date'] = df['Original Date'].apply(safe_convert_date)

# Function to process Date changes
def process_dates(row, current_date):
    original_date = row.get('Original Date')
    date = row.get('Date')
    
    # Check if dates are valid
    if pd.isna(date) or (original_date and (original_date.year < current_date.year or original_date.month < current_date.month)):
        return current_date.replace(day=1) - pd.Timedelta(days=1)  # Last day of current month
    return date

# Get today's date
today = pd.Timestamp.now()
current_month_last_date = today.replace(day=1) + pd.offsets.MonthEnd(0)

# Apply logic to update Date column and highlight changes
for index, row in df.iterrows():
    updated_date = process_dates(row, current_month_last_date)
    if updated_date != row['Date']:  # Track changes
        df.at[index, 'Date'] = updated_date
        ws.cell(row=index + 2, column=df.columns.get_loc('Date') + 1).value = updated_date  # Update cell value
        ws.cell(row=index + 2, column=df.columns.get_loc('Date') + 1).fill = yellow_fill  # Highlight updated cell

# Handle 'Type' and 'Description' specific logic only if the columns exist
if 'Type' in df.columns:
    # Example logic for 'Type'
    df['Type'] = df['Type'].apply(lambda x: x.upper() if pd.notna(x) else "")
if 'Description' in df.columns:
    # Example logic for 'Description'
    df['Description'] = df['Description'].fillna("No description available")

# Save the updated Excel file
wb.save("updated_file.xlsx")


In [ ]:
import pandas as pd
import math
import datetime

# Example source data
data = {
    "FTE": [6.5, -12.3, 3, 0, None, 10],
    "Type": ["BC-TT", "Investments", "Saves", "Other", "Saves", "BC-TT"],
    "GM": [0, 0, 0, 0, 0, 1],
    "MD": [0, 0, 0, 0, 0, 0],
    "3": [0, 0, 0, 0, 0, 2],
    "4": [2.5, -4, 0, 0, 0, 0],
    "5": [0, 0, 0, 0, 0, 0],
    "6": [3, -6, 0, 0, 0, 3],
    "7": [1, -1, 1, 0, 0, 4],
    "8": [0, -1.3, 0, 0, 0, 0],
    "Entity": ["Entity1", "Entity2", "Entity3", "Entity4", "Entity5", "Entity6"],
    "Description": ["Example 1", "Example 2", "Example 3", "Example 4", "Example 5", "Example 6"],
    "Country": ["Country1", "Country2", "Country3", "Country4", "Country5", "Country6"],
    "RTN": ["RTN 123 - Example", "RTN 456 - Sample", "RTN 789 - Test", "RTN 101 - Case", "RTN 112 - Trial", "RTN 113 - Check"]
}

source_df = pd.DataFrame(data)

# Columns to check for GCB values
gcb_columns = ["GM", "MD", "3", "4", "5", "6", "7", "8"]

# Clean and convert FTE column
source_df["FTE"] = pd.to_numeric(source_df["FTE"], errors="coerce")

# Filter rows where FTE is valid and does not equal 0
filtered_df = source_df[
    source_df["FTE"].notnull() & (source_df["FTE"] != 0) & (source_df.get("Type", "").isin(["BC-TT", "Investments", "Saves"]))
]

# Create the template dataframe with required columns
template_columns = [
    "Big Grid Stack", "AOP YEAR", "Business Framework RTN Code",
    "Business Framework", "Business Framework Group", "Country",
    "GCB", "Emp. Type", "Hiring Source", "FTE", "Start Month",
    "WPB", "GBM", "CMB", "Diver", "Entity", "Description"
]
template_df = pd.DataFrame(columns=template_columns)

# Process each row in the filtered dataframe
for _, row in filtered_df.iterrows():
    fte_value = row["FTE"]
    duplicate_fte = -1 if fte_value < 0 else 1  # Determine positive or negative FTE
    abs_fte = abs(fte_value)
    int_part = math.floor(abs_fte)  # Integer part of FTE
    fractional_part = abs_fte - int_part  # Fractional part of FTE

    # Calculate GCB distribution
    gcb_distribution = []
    for col in gcb_columns:
        value = pd.to_numeric(row.get(col), errors="coerce")  # Convert to numeric, handle non-numeric gracefully
        if not pd.isna(value) and value != 0:
            gcb_distribution.extend([col] * abs(math.ceil(value)))

    # Ensure GCB distribution matches the FTE rows
    total_required_rows = int_part + (1 if fractional_part > 0 else 0)
    if len(gcb_distribution) < total_required_rows:
        gcb_distribution.extend(["na"] * (total_required_rows - len(gcb_distribution)))

    # Dynamically calculate Start Month
    current_year = 2025
    if fte_value > 0:
        start_month = datetime.date(current_year, 1, 31)  # Example logic for positive FTE
    elif fte_value < 0:
        start_month = datetime.date(current_year, 2, 28)  # Example logic for negative FTE
    else:
        start_month = datetime.date(current_year, 12, 31)  # Default fallback

    # Create rows in the template
    for i in range(total_required_rows):
        current_fte = (
            fractional_part if i == total_required_rows - 1 and fractional_part > 0 else 1
        ) * duplicate_fte

        big_grid_stack = (
            "New Perm Position (within FRP) - Staff Drawdown" if current_fte > 0
            else "Forecast program/Other Saves-Saves Forecast Tracker"
        )
        if row.get("Type") == "BC-TT":
            big_grid_stack = "Inter Boundary Changes"

        new_row = {
            "Big Grid Stack": big_grid_stack,
            "AOP YEAR": "FY25",
            "Business Framework RTN Code": row["RTN"].split()[0],
            "Business Framework": "",
            "Business Framework Group": "",
            "Country": row.get("Country", ""),
            "GCB": gcb_distribution[i],
            "Emp. Type": "FTE",
            "Hiring Source": "External",
            "FTE": current_fte,
            "Start Month": start_month.strftime("%m/%d/%Y"),
            "WPB": "0%",
            "GBM": "100.00%",
            "CMB": "0%",
            "Diver": "",
            "Entity": row.get("Entity", ""),
            "Description": row.get("Description", "No description available"),
        }
        template_df = pd.concat([template_df, pd.DataFrame([new_row])], ignore_index=True)

# Output the result
print(template_df)


In [ ]:
import pandas as pd
import datetime
import calendar
from openpyxl import load_workbook
from openpyxl.styles import PatternFill

def adjust_and_highlight_dates(input_file, output_file):
    # Read the Excel file into a dataframe
    df = pd.read_excel(input_file)

    # Get today's date, current year, and current month
    today = datetime.datetime.today()
    current_year = today.year
    current_month = today.month

    # Convert the Date column to datetime format
    df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y', errors='coerce')

    # Preserve the original dates for comparison
    df['Original Date'] = df['Date']

    # Iterate through the dataframe and update dates if necessary
    for index, row in df.iterrows():
        original_date = row['Original Date']
        if pd.notna(original_date):  # Ensure the date is valid
            # Check if the year is not current or the month is before the current month
            if original_date.year != current_year or original_date.month < current_month:
                # Get the last day of the current month
                last_day_of_month = datetime.date(current_year, current_month, calendar.monthrange(current_year, current_month)[1])
                df.at[index, 'Date'] = last_day_of_month  # Update the date

    # Save the updated dataframe to the Excel file
    df.to_excel(output_file, index=False)

    # Open the workbook to apply formatting
    wb = load_workbook(output_file)
    ws = wb.active

    # Apply yellow highlight to cells in the "Date" column where changes were made
    yellow_fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")
    for index, row in df.iterrows():
        if row['Original Date'] != row['Date']:  # Compare original and updated date
            excel_row = index + 2  # Offset for Excel's 1-based index and header row
            ws.cell(row=excel_row, column=2).fill = yellow_fill  # Column 2 corresponds to the 'Date' column

    # Save the workbook with highlighted changes
    wb.save(output_file)
    print("Dates adjusted and changes highlighted where necessary.")

# Example usage
input_file = "input_dates.xlsx"  # Replace with your input file
output_file = "output_dates.xlsx"  # Replace with your output file
adjust_and_highlight_dates(input_file, output_file)


In [ ]:
import openpyxl
from openpyxl.styles import Alignment, Font, PatternFill
from openpyxl.utils import get_column_letter

def format_excel_with_headers(file_path):
    # Load the workbook
    workbook = openpyxl.load_workbook(file_path)
    
    for sheet_name in workbook.sheetnames:
        sheet = workbook[sheet_name]
        
        # Insert the sheet name as a heading in A1
        sheet.insert_rows(1)
        sheet['A1'] = sheet_name
        sheet.merge_cells(start_row=1, start_column=1, end_row=1, end_column=sheet.max_column)
        sheet['A1'].font = Font(size=14, bold=True)
        sheet['A1'].alignment = Alignment(horizontal="center", vertical="center")
        sheet['A1'].fill = PatternFill(start_color="B0C4DE", end_color="B0C4DE", fill_type="solid")  # Light Blue
        
        # Apply formatting to the header row (now at row 2)
        header_row = 2
        for col in range(1, sheet.max_column + 1):
            cell = sheet.cell(row=header_row, column=col)
            cell.font = Font(bold=True)
            cell.fill = PatternFill(start_color="ADD8E6", end_color="ADD8E6", fill_type="solid")  # Light Blue
            cell.alignment = Alignment(horizontal="center", vertical="center")
        
        # Set column widths
        for col in range(1, sheet.max_column + 1):
            max_length = 0
            column = get_column_letter(col)
            for row in range(1, sheet.max_row + 1):
                cell = sheet[f"{column}{row}"]
                try:
                    if cell.value:
                        max_length = max(max_length, len(str(cell.value)))
                except Exception:
                    pass
            adjusted_width = max_length + 2
            sheet.column_dimensions[column].width = adjusted_width

    # Save the updated workbook
    workbook.save(file_path)
    print(f"Workbook formatted and saved at: {file_path}")

# Example Usage
file_path = "organization_structure.xlsx"
format_excel_with_headers(file_path)


In [ ]:
from pptx import Presentation
from pptx.util import Inches, Pt
import openpyxl

def add_table_slide(presentation, sheet_name, sheet):
    slide = presentation.slides.add_slide(presentation.slide_layouts[5])  # Blank slide
    title = slide.shapes.title
    title.text = f"Data for {sheet_name}"

    # Get the data
    data = [[cell.value for cell in row] for row in sheet.iter_rows(values_only=True)]
    rows, cols = len(data), len(data[0])

    # Add table
    left = Inches(1.0)
    top = Inches(1.5)
    width = Inches(8.0)
    height = Inches(5.0)
    table = slide.shapes.add_table(rows, cols, left, top, width, height).table

    # Fill the table with data
    for i, row in enumerate(data):
        for j, value in enumerate(row):
            cell = table.cell(i, j)
            cell.text = str(value) if value else ""
            cell.text_frame.paragraphs[0].font.size = Pt(10)

    # Format header row
    for cell in table.rows[0].cells:
        cell.text_frame.paragraphs[0].font.bold = True
        cell.fill.solid()
        cell.fill.fore_color.rgb = RGBColor(173, 216, 230)  # Light Blue

def generate_table_ppt(excel_file, ppt_file):
    # Load the Excel workbook
    workbook = openpyxl.load_workbook(excel_file)
    presentation = Presentation()

    # Iterate through sheets and process data
    for sheet_name in workbook.sheetnames:
        sheet = workbook[sheet_name]
        add_table_slide(presentation, sheet_name, sheet)

    # Save the PowerPoint
    presentation.save(ppt_file)
    print(f"PPT saved as {ppt_file}")

# Example Usage
input_excel = "formatted_organization_structure.xlsx"
output_ppt = "organization_structure_table.pptx"
generate_table_ppt(input_excel, output_ppt)


In [ ]:
from pptx import Presentation
from pptx.enum.shapes import MSO_SHAPE
from pptx.util import Inches, Pt
import openpyxl

def add_org_chart_slide(presentation, sheet_name, hierarchy):
    slide = presentation.slides.add_slide(presentation.slide_layouts[5])  # Blank slide
    title = slide.shapes.title
    title.text = f"Organizational Chart: {sheet_name}"

    # Add shapes for hierarchy
    top_margin = 1.5  # Inches
    left_margin = 1.0  # Inches
    shape_width = 2.0  # Inches
    shape_height = 1.0  # Inches
    level_gap = 1.5  # Vertical gap between levels
    horizontal_gap = 2.5  # Horizontal gap between shapes

    # Recursive function to draw hierarchy
    def draw_hierarchy(manager, x, y, slide):
        shape = slide.shapes.add_shape(
            MSO_SHAPE.RECTANGLE,
            Inches(x), Inches(y),
            Inches(shape_width), Inches(shape_height)
        )
        shape.text = manager
        shape.text_frame.paragraphs[0].font.size = Pt(12)
        return shape

    y = top_margin
    for manager1, sub_hierarchy in hierarchy.items():
        shape1 = draw_hierarchy(manager1, left_margin, y, slide)
        x = left_margin
        for manager2, employees in sub_hierarchy.items():
            y += level_gap
            shape2 = draw_hierarchy(manager2, x + horizontal_gap, y, slide)
            for i, employee in enumerate(employees):
                draw_hierarchy(employee, x + horizontal_gap * (i + 2), y + level_gap, slide)

def generate_org_chart_ppt(excel_file, ppt_file):
    # Load the Excel workbook
    workbook = openpyxl.load_workbook(excel_file)
    presentation = Presentation()

    # Iterate through sheets and process data
    for sheet_name in workbook.sheetnames:
        sheet = workbook[sheet_name]
        hierarchy = extract_hierarchy(sheet)
        add_org_chart_slide(presentation, sheet_name, hierarchy)

    # Save the PowerPoint
    presentation.save(ppt_file)
    print(f"PPT saved as {ppt_file}")

# Example Usage
input_excel = "formatted_organization_structure.xlsx"
output_ppt = "organization_structure_chart.pptx"
generate_org_chart_ppt(input_excel, output_ppt)


In [ ]:
import openpyxl
from pptx import Presentation
from pptx.util import Pt
from pptx.dml.color import RGBColor

def extract_hierarchy(sheet):
    hierarchy = {}
    for row in sheet.iter_rows(min_row=3, values_only=True):  # Start reading data from row 3
        level1, level2, level3 = row[0:3], row[3:6], row[6:9]
        manager1 = " | ".join([str(x) for x in level1 if x])
        manager2 = " | ".join([str(x) for x in level2 if x])
        employee = " | ".join([str(x) for x in level3 if x])

        if manager1 not in hierarchy:
            hierarchy[manager1] = {}
        if manager2 not in hierarchy[manager1]:
            hierarchy[manager1][manager2] = []
        hierarchy[manager1][manager2].append(employee)

    return hierarchy

def add_slide_with_chart(presentation, sheet_name, hierarchy):
    slide = presentation.slides.add_slide(presentation.slide_layouts[5])  # Blank slide
    # Add title
    title = slide.shapes.title
    title.text = sheet_name
    title.text_frame.paragraphs[0].font.bold = True
    title.text_frame.paragraphs[0].font.size = Pt(24)

    # Add text box for the org chart
    left = Pt(50)
    top = Pt(100)
    width = Pt(800)
    height = Pt(500)
    text_box = slide.shapes.add_textbox(left, top, width, height)
    text_frame = text_box.text_frame
    text_frame.word_wrap = True
    text_frame.margin_left = Pt(10)
    text_frame.margin_top = Pt(10)

    # Build the org chart text
    text_frame.text = f"Organizational Chart for {sheet_name}\n"
    for manager1, sub_hierarchy in hierarchy.items():
        p = text_frame.add_paragraph()
        p.text = manager1
        p.font.bold = True
        p.font.size = Pt(16)
        p.font.color.rgb = RGBColor(0, 51, 102)  # Dark blue

        for manager2, employees in sub_hierarchy.items():
            p2 = text_frame.add_paragraph()
            p2.text = f"    {manager2}"
            p2.font.bold = False
            p2.font.size = Pt(14)
            p2.font.color.rgb = RGBColor(0, 102, 0)  # Green

            for employee in employees:
                p3 = text_frame.add_paragraph()
                p3.text = f"        {employee}"
                p3.font.size = Pt(12)
                p3.font.color.rgb = RGBColor(0, 0, 0)  # Black

def generate_organizational_ppt(excel_file, ppt_file):
    # Load the Excel workbook
    workbook = openpyxl.load_workbook(excel_file)
    presentation = Presentation()

    # Iterate through sheets and process data
    for sheet_name in workbook.sheetnames:
        sheet = workbook[sheet_name]
        hierarchy = extract_hierarchy(sheet)
        add_slide_with_chart(presentation, sheet_name, hierarchy)

    # Save the PowerPoint
    presentation.save(ppt_file)
    print(f"PPT saved as {ppt_file}")

# Input Excel and output PPT file paths
input_excel = "formatted_organization_structure.xlsx"
output_ppt = "organization_structure_presentation.pptx"

# Generate the presentation
generate_organizational_ppt(input_excel, output_ppt)


In [ ]:
from openpyxl import load_workbook
from openpyxl.styles import Font, Alignment, PatternFill
from openpyxl.utils import get_column_letter

def format_sheet(sheet):
    # Set the sheet name as the title in cell A1
    sheet_title = sheet.title
    sheet.merge_cells('A1:I1')  # Adjust range based on your data width
    sheet['A1'] = sheet_title
    sheet['A1'].font = Font(bold=True, size=14)
    sheet['A1'].alignment = Alignment(horizontal='center')

    # Style the header row
    header_fill = PatternFill(start_color="ADD8E6", end_color="ADD8E6", fill_type="solid")  # Light blue background
    header_font = Font(bold=True)
    header_alignment = Alignment(horizontal='center', vertical='center')

    # Assuming row 2 is the header row
    for cell in sheet[2]:  # Header row is the second row
        cell.fill = header_fill
        cell.font = header_font
        cell.alignment = header_alignment

    # Set fixed column width
    fixed_width = 20  # Set desired width
    for col in sheet.columns:
        col_letter = get_column_letter(col[0].column)  # Get column letter
        sheet.column_dimensions[col_letter].width = fixed_width

def format_workbook(file_path, output_path):
    # Load the workbook
    wb = load_workbook(file_path)

    # Format each sheet
    for sheet in wb.worksheets:
        format_sheet(sheet)

    # Save the formatted workbook
    wb.save(output_path)
    print(f"Formatted file saved as: {output_path}")

# Input and output file paths
input_file = "organization_structure.xlsx"
output_file = "formatted_organization_structure.xlsx"

# Call the function to format the workbook
format_workbook(input_file, output_file)


In [ ]:
import pandas as pd
from openpyxl import Workbook

# Load the input Excel file
input_file = "your_input_file.xlsx"  # Replace with your input file path
output_file = "organization_structure.xlsx"  # Output file path

# Read the data
df = pd.read_excel(input_file)

# Ensure consistent column names (case-insensitive matching)
df.columns = df.columns.str.strip()

# Unique countries
countries = df['Work Location Country/Territory Name'].dropna().unique()

# Create a new workbook
wb = Workbook()
wb.remove(wb.active)  # Remove the default sheet

# Function to find subordinates
def find_subordinates(df, manager_name):
    return df[df['Entity Manager Employee Name'] == manager_name]

# Process each country
for country in countries:
    # Filter data for the country
    country_df = df[df['Work Location Country/Territory Name'] == country]
    # Get unique BF Level 4 Names for the country
    bf_level_4_names = country_df['BF Level 4 Name'].dropna().unique()

    # Create a sheet for this country
    for bf_level_4_name in bf_level_4_names:
        # Filter data for the BF Level 4 Name
        bf_df = country_df[country_df['BF Level 4 Name'] == bf_level_4_name]

        # Sort data by Global Career Band (MD > 3 > 4 > ... > 8)
        bf_df['Global Career Band'] = bf_df['Global Career Band'].astype(str)  # Ensure values are strings
        sorted_bf_df = bf_df.sort_values('Global Career Band', key=lambda col: col.map(lambda x: 'MD' if x == 'MD' else int(x) if x.isdigit() else 999))

        # Prepare data for three levels
        rows = []
        for _, level_1 in sorted_bf_df.iterrows():
            level_1_details = [level_1['Employee Name'], level_1['Global Career Band'], level_1['Position Title']]
            level_2_df = find_subordinates(df, level_1['Employee Name'])

            if not level_2_df.empty:
                first_level_1 = True  # To track if Level 1 details have been added
                for _, level_2 in level_2_df.iterrows():
                    level_2_details = [level_2['Employee Name'], level_2['Global Career Band'], level_2['Position Title']]
                    level_3_df = find_subordinates(df, level_2['Employee Name'])

                    if not level_3_df.empty:
                        first_level_2 = True  # To track if Level 2 details have been added
                        for _, level_3 in level_3_df.iterrows():
                            level_3_details = [level_3['Employee Name'], level_3['Global Career Band'], level_3['Position Title']]
                            rows.append((level_1_details if first_level_1 else ["", "", ""]) +
                                        (level_2_details if first_level_2 else ["", "", ""]) +
                                        level_3_details)
                            first_level_1 = False
                            first_level_2 = False
                    else:
                        rows.append((level_1_details if first_level_1 else ["", "", ""]) + level_2_details + ["", "", ""])
                        first_level_1 = False
            else:
                rows.append(level_1_details + ["", "", ""] + ["", "", ""])

        # Write data to the sheet
        sheet_name = f"{bf_level_4_name} - {country}"
        sheet_name = sheet_name[:31]  # Excel sheet names have a max length of 31
        sheet = wb.create_sheet(title=sheet_name)
        # Write headers
        sheet.append(["Level 1 Employee Name", "Global Career Band", "Position Title",
                      "Level 2 Employee Name", "Global Career Band", "Position Title",
                      "Level 3 Employee Name", "Global Career Band", "Position Title"])
        # Write rows
        for row in rows:
            sheet.append(row)

# Save the workbook
wb.save(output_file)

print(f"Organization structure saved to {output_file}")


In [1]:
import pandas as pd
import math
import datetime

# Example source data
data = {
    "FTE": [6.5, -12.3, 3, 0, None, 10],
    "Type": ["BC-TT", "Investments", "Saves", "Other", "Saves", "BC-TT"],
    "GM": [0, 0, 0, 0, 0, 1],
    "MD": [0, 0, 0, 0, 0, 0],
    "3": [0, 0, 0, 0, 0, 2],
    "4": [2.5, -4, 0, 0, 0, 0],
    "5": [0, 0, 0, 0, 0, 0],
    "6": [3, -6, 0, 0, 0, 3],
    "7": [1, -1, 1, 0, 0, 4],
    "8": [0, -1.3, 0, 0, 0, 0],
    "Entity": ["Entity1", "Entity2", "Entity3", "Entity4", "Entity5", "Entity6"],
    "Description": ["Example 1", "Example 2", "Example 3", "Example 4", "Example 5", "Example 6"],
    "Country": ["Country1", "Country2", "Country3", "Country4", "Country5", "Country6"],
    "RTN": ["RTN 123 - Example", "RTN 456 - Sample", "RTN 789 - Test", "RTN 101 - Case", "RTN 112 - Trial", "RTN 113 - Check"]
}

source_df = pd.DataFrame(data)

# Columns to check for GCB values
gcb_columns = ["GM", "MD", "3", "4", "5", "6", "7", "8"]

# Clean and convert FTE column
source_df["FTE"] = pd.to_numeric(source_df["FTE"], errors="coerce")

# Filter rows where FTE is valid and does not equal 0
filtered_df = source_df[
    source_df["FTE"].notnull() & (source_df["FTE"] != 0) & (source_df["Type"].isin(["BC-TT", "Investments", "Saves"]))
]

# Create the template dataframe with required columns
template_columns = [
    "Big Grid Stack", "AOP YEAR", "Business Framework RTN Code",
    "Business Framework", "Business Framework Group", "Country",
    "GCB", "Emp. Type", "Hiring Source", "FTE", "Start Month",
    "WPB", "GBM", "CMB", "Diver", "Entity", "Description"
]
template_df = pd.DataFrame(columns=template_columns)

# Process each row in the filtered dataframe
for _, row in filtered_df.iterrows():
    fte_value = row["FTE"]
    duplicate_fte = -1 if fte_value < 0 else 1  # Determine positive or negative FTE
    abs_fte = abs(fte_value)
    int_part = math.floor(abs_fte)  # Integer part of FTE
    fractional_part = abs_fte - int_part  # Fractional part of FTE

    # Calculate GCB distribution
    gcb_distribution = []
    for col in gcb_columns:
        value = pd.to_numeric(row[col], errors="coerce")  # Convert to numeric, handle non-numeric gracefully
        if not pd.isna(value) and value != 0:
            gcb_distribution.extend([col] * abs(math.ceil(value)))

    # Ensure GCB distribution matches the FTE rows
    total_required_rows = int_part + (1 if fractional_part > 0 else 0)
    if len(gcb_distribution) < total_required_rows:
        gcb_distribution.extend(["na"] * (total_required_rows - len(gcb_distribution)))

    # Dynamically calculate Start Month
    current_year = 2025
    if fte_value > 0:
        start_month = datetime.date(current_year, 1, 31).strftime("%m/%d/%Y")  # Example logic for positive FTE
    elif fte_value < 0:
        start_month = datetime.date(current_year, 2, 28).strftime("%m/%d/%Y")  # Example logic for negative FTE
    else:
        start_month = datetime.date(current_year, 12, 31).strftime("%m/%d/%Y")  # Default fallback

    # Create rows in the template
    for i in range(total_required_rows):
        current_fte = (
            fractional_part if i == total_required_rows - 1 and fractional_part > 0 else 1
        ) * duplicate_fte

        big_grid_stack = (
            "New Perm Position (within FRP) - Staff Drawdown" if current_fte > 0
            else "Forecast program/Other Saves-Saves Forecast Tracker"
        )
        if row["Type"] == "BC-TT":
            big_grid_stack = "Inter Boundary Changes"

        new_row = {
            "Big Grid Stack": big_grid_stack,
            "AOP YEAR": "FY25",
            "Business Framework RTN Code": row["RTN"].split()[0],
            "Business Framework": "",
            "Business Framework Group": "",
            "Country": row["Country"],
            "GCB": gcb_distribution[i],
            "Emp. Type": "FTE",
            "Hiring Source": "External",
            "FTE": current_fte,
            "Start Month": start_month,
            "WPB": "0%",
            "GBM": "100.00%",
            "CMB": "0%",
            "Diver": "",
            "Entity": row["Entity"],
            "Description": row["Description"],
        }
        template_df = pd.concat([template_df, pd.DataFrame([new_row])], ignore_index=True)

# Output the result
print(template_df)


                                       Big Grid Stack AOP YEAR  \
0     New Perm Position (within FRP) - Staff Drawdown     FY25   
1     New Perm Position (within FRP) - Staff Drawdown     FY25   
2     New Perm Position (within FRP) - Staff Drawdown     FY25   
3     New Perm Position (within FRP) - Staff Drawdown     FY25   
4     New Perm Position (within FRP) - Staff Drawdown     FY25   
5     New Perm Position (within FRP) - Staff Drawdown     FY25   
6     New Perm Position (within FRP) - Staff Drawdown     FY25   
7   Forecast program/Other Saves-Saves Forecast Tr...     FY25   
8   Forecast program/Other Saves-Saves Forecast Tr...     FY25   
9   Forecast program/Other Saves-Saves Forecast Tr...     FY25   
10  Forecast program/Other Saves-Saves Forecast Tr...     FY25   
11  Forecast program/Other Saves-Saves Forecast Tr...     FY25   
12  Forecast program/Other Saves-Saves Forecast Tr...     FY25   
13  Forecast program/Other Saves-Saves Forecast Tr...     FY25   
14  Foreca

In [ ]:
import os
import pandas as pd

def process_hdpi_file(input_file, month_name):
    """
    Process the HDPI Excel file.
    Modify this function to include your specific processing logic.
    """
    # Load the Excel file
    df = pd.read_excel(input_file)

    # Perform your processing logic here
    # Example: Add a new column for demonstration
    df["Processed"] = f"Processed for {month_name}"

    return df

def process_files_in_folder(ifolder, ofolder):
    """
    Processes all Excel files in the given folder.
    """
    # Ensure output folder exists
    os.makedirs(ofolder, exist_ok=True)

    # Get the list of files in the input folder
    files = [f for f in os.listdir(ifolder) if f.endswith(".xlsx")]

    for file_name in files:
        try:
            # Extract the file path
            input_file_path = os.path.join(ifolder, file_name)

            # Extract the month name (last three letters before ".xlsx")
            month_name = file_name.split()[-1][:3]

            # Load and process the file
            processed_df = process_hdpi_file(input_file_path, month_name)

            # Save the processed file with the desired name in the output folder
            output_file_name = f"HDPI {month_name}.xlsx"
            output_file_path = os.path.join(ofolder, output_file_name)
            processed_df.to_excel(output_file_path, index=False)

            print(f"Processed and saved: {output_file_path}")
        except Exception as e:
            print(f"Error processing {file_name}: {e}")

# Define input and output folder paths
ifolder = r"C:\path\to\input\folder"  # Replace with your input folder path
ofolder = r"C:\path\to\output\folder"  # Replace with your output folder path

# Process all files in the folder
process_files_in_folder(ifolder, ofolder)


In [7]:
import win32com.client

def convert_docx_to_pdf(docx_path, pdf_path):
    # Initialize Word application
    word = win32com.client.Dispatch("Word.Application")
    word.Visible = False  # Run in the background

    # Open the .docx file
    doc = word.Documents.Open(docx_path)

    # Save as PDF
    doc.SaveAs(pdf_path, FileFormat=17)  # 17 corresponds to the PDF format

    # Close the document and quit Word
    doc.Close()
    word.Quit()

# Example usage
docx_file = r"C:\\Users\\KS\\Thesis.docx"
pdf_file = r"C:\\Users\\KS\\Thesis_test.pdf"
convert_docx_to_pdf(docx_file, pdf_file)


com_error: (-2147352567, 'Exception occurred.', (0, 'Microsoft Word', 'Command failed', 'wdmain11.chm', 36966, -2146824090), None)

In [27]:
import pandas as pd
from openpyxl import load_workbook

# Sample data for spocdf and olddf (replace these with actual data loading as needed)
spocdf = pd.DataFrame({
    'Billing Contact Name': ['Alice', 'Alice', 'Bob', 'Alice', 'Charlie', 'Charlie'],
    'Billing Contact PS ID': [101, 102, 201, 103, 301, 301]
})

olddf = pd.DataFrame({
    'Billing Contact Name': ['Alice', 'Bob', 'Charlie'],
    'Billing Contact PS ID': [101, 201, 301]  # Assuming these are the correct PS IDs
})

# Step 1: Merge spocdf with olddf to add the correct PS ID as a reference
merged_df = spocdf.merge(
    olddf,
    on='Billing Contact Name',
    how='left',
    suffixes=('', '_correct')
)

# Step 2: Filter for rows where the 'Billing Contact PS ID' does not match the 'Correct Billing Contact PS ID'
# Retain only records where there is a mismatch
mismatched_psid_df = merged_df[merged_df['Billing Contact PS ID'] != merged_df['Billing Contact PS ID_correct']].copy()

# Rename the correct column for clarity
mismatched_psid_df.rename(columns={'Billing Contact PS ID_correct': 'Correct Billing Contact PS ID'}, inplace=True)

# # Step 3: Write the filtered DataFrame with mismatched PS IDs to a new sheet in the workbook
# file_path = 'your_workbook.xlsx'  # Replace with your actual file path
# workbook = load_workbook(file_path)

# # Check if 'Duplicate PSID' sheet exists, and delete if it does
# if 'Duplicate PSID' in workbook.sheetnames:
#     del workbook['Duplicate PSID']
# workbook.save(file_path)  # Save after deleting to ensure it's applied

# # Append the new data to the workbook in 'Duplicate PSID'
# with pd.ExcelWriter(file_path, engine='openpyxl', mode='a') as writer:
#     mismatched_psid_df.to_excel(writer, sheet_name='Duplicate PSID', index=False)
print(mismatched_psid_df)
print("Filtered Duplicate PSID sheet with mismatched PS IDs has been created and saved.")


  Billing Contact Name  Billing Contact PS ID  Correct Billing Contact PS ID
1                Alice                    102                            101
3                Alice                    103                            101
Filtered Duplicate PSID sheet with mismatched PS IDs has been created and saved.


In [19]:
import pandas as pd
import numpy as np

# Sample DataFrames
df1 = pd.DataFrame({
    'A2': [101, 102, 103, 104],
    'C': ['new', 'old', 'new', 'old'],
    'SC': [None, 'value2', None, 'value4']
})

df2 = pd.DataFrame({
    'AA2': [101, 102, 103, 104],
    'SC': ['lookup_value1', 'lookup_value2', 'lookup_value3', 'lookup_value4']
})

# Step 1: Create a mask for rows where 'C' is 'new' and 'SC' is blank (None or NaN)
mask = (df1['C'] == 'new') & (df1['SC'].isna())

# Step 2: Create a lookup dictionary from df2 for easy access
lookup_dict = df2.set_index('AA2')['SC'].to_dict()

# Step 3: Use np.where() to update 'SC' column conditionally
df1['SC'] = np.where(
    mask,  # Only update where the condition is True
    df1['A2'].map(lookup_dict).fillna(df1['SC']),  # Map A2 to the lookup dict and fill NaN with existing SC
    df1['SC']  # Retain the existing SC values
)

# Output the updated df1
print(df1)


    A2    C             SC
0  101  new  lookup_value1
1  102  old         value2
2  103  new  lookup_value3
3  104  old         value4


In [1]:
import pandas as pd

# Sample DataFrame 1 (df1)
data1 = {'A': [101, 102, 103, 104],
         'C': ['X1', 'X2', 'X3', 'X4'],
         'E': ['Y11', 'Y2', 'Y1', 'Y4'],
         'D': [None, None, None, None]}  # Initially empty Column D
df1 = pd.DataFrame(data1)
print(df1)
# Sample DataFrame 2 (df2)
data2 = {'A': [101, 103, 105, 106],
         'C': ['X1', 'X3', 'X5', 'X6'],
         'E': ['Y11', 'Y2', 'Y4', 'Y1'],
         'LookupValue': ['Val1', 'Val3', 'Val5', 'Val6']}  # Column with values to fetch
df2 = pd.DataFrame(data2)
print(df2)

# Step 1: First Lookup - Based on Column 'C'
df1 = pd.merge(df1, df2[['C', 'LookupValue']], how='left', on='C')  # Merge on 'C'
df1['D'] = df1['D'].mask(df1['D'].isna(), df1['LookupValue'])  # Use `mask` to fill NaN in 'D' only
df1.drop(columns=['LookupValue'], inplace=True)  # Drop temporary column

# Step 2: Second Lookup - Based on Column 'E'
df1 = pd.merge(df1, df2[['E', 'LookupValue']], how='left', on='E')  # Merge on 'E'
df1['D'] = df1['D'].mask(df1['D'].isna(), df1['LookupValue'])  # Update only NaNs in 'D'
df1.drop(columns=['LookupValue'], inplace=True)  # Drop temporary column

# Step 3: Third Lookup - Based on Column 'A'
df1 = pd.merge(df1, df2[['A', 'LookupValue']], how='left', on='A')  # Merge on 'A'
df1['D'] = df1['D'].mask(df1['D'].isna(), df1['LookupValue'])  # Update only remaining NaNs in 'D'
df1.drop(columns=['LookupValue'], inplace=True)  # Drop temporary column

# Final Output
print(df1)


     A   C    E     D
0  101  X1  Y11  None
1  102  X2   Y2  None
2  103  X3   Y1  None
3  104  X4   Y4  None
     A   C    E LookupValue
0  101  X1  Y11        Val1
1  103  X3   Y2        Val3
2  105  X5   Y4        Val5
3  106  X6   Y1        Val6
     A   C    E     D
0  101  X1  Y11  Val1
1  102  X2   Y2  Val3
2  103  X3   Y1  Val3
3  104  X4   Y4  Val5


In [2]:
import pandas as pd
import os
from datetime import datetime

# Function to generate columns for all months of the year
def generate_monthly_columns(year_suffix, num_years=1):
    months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
    columns = []
    
    for i in range(num_years):
        for month in months:
            columns.append(f"{month} {year_suffix + i}")
    
    return columns

# Function to update dataframe with monthly data from files
def update_monthly_data(df, folder_path, year_suffix):
    months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
    current_month = datetime.now().month  # Get the current month (1-12)
    
    for month_idx in range(current_month):
        month_name = months[month_idx]  # Get month name
        col_name = f"{month_name} {year_suffix}"  # Column name to update
        
        # Read corresponding file for the current month if exists
        file_path = os.path.join(folder_path, f"{month_name}_{year_suffix}.csv")  # Assuming CSV format
        
        if os.path.exists(file_path):
            monthly_data = pd.read_csv(file_path)  # Adjust if files are in Excel
            
            # Assuming you have a common column like 'Position ID' to merge data
            if 'Position ID' in monthly_data.columns:
                # Merge or update existing data based on 'Position ID'
                df = df.merge(monthly_data[['Position ID', 'Value']], on='Position ID', how='left')
                
                # Assign merged 'Value' to the corresponding column (e.g., "Jan 24")
                df[col_name] = df['Value']
                df = df.drop(columns=['Value'])  # Remove the temporary 'Value' column
    
    return df

# Step 1: Generate columns for one year
columns_2024 = generate_monthly_columns(24)

# Step 2: Create an empty dataframe with 'Position ID' and the generated columns
df = pd.DataFrame(columns=['Position ID'] + columns_2024)

# Step 3: Simulate loading data from previous step (e.g., from GHA)
# Assuming 'Position ID' column is already there
# df = pd.read_csv('previous_step_data.csv')  # Or however you load it

# Step 4: Update DataFrame with monthly data up to the current month
folder_path = 'path_to_your_files'  # Specify the folder where monthly files are stored
df = update_monthly_data(df, folder_path, 24)

# Step 5: Save the final updated DataFrame to Excel or CSV
df.to_csv('final_output.csv', index=False)  # Adjust for Excel if needed


FileNotFoundError: [Errno 2] No such file or directory: 'jan_2024_data.csv'

In [1]:
import pandas as pd
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.worksheet.datavalidation import DataValidation

# Sample dataframe
data = {'Name': ['Alice', 'Bob', 'Charlie'],
        'Age': [24, 30, 22],
        'Gender': ['Female', 'Male', 'Male']}  # We will apply drop-down for Gender

df = pd.DataFrame(data)

# Save DataFrame to an Excel file
output_path = 'output_with_dropdown.xlsx'

# Create a workbook and add the dataframe to the sheet
wb = Workbook()
ws = wb.active

# Write the DataFrame to the worksheet
for row in dataframe_to_rows(df, index=False, header=True):
    ws.append(row)

# Define the list of allowed values for the drop-down
drop_down_list = ['Male', 'Female', 'Other']

# Create a DataValidation object for the drop-down list
dv = DataValidation(type="list", formula1=f'"{",".join(drop_down_list)}"', showDropDown=False)

# Add the data validation to the 'Gender' column (assuming it is in column C, starting from C2)
ws.add_data_validation(dv)
dv.add(f"C2:C{len(df) + 1}")  # Apply validation to the Gender column for all rows

# Save the workbook
wb.save(output_path)

print(f"Excel file saved with drop-down in column 'Gender'.")


Excel file saved with drop-down in column 'Gender'.


In [15]:
import pandas as pd
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.worksheet.datavalidation import DataValidation

# Sample dataframe
data = {'Name': ['Alice', 'Bob', 'Charlie'],
        'Age': [24, 30, 22],
        'Gender': ['Female', 'Male', '']}  # Leave empty so users can select from the dropdown

df = pd.DataFrame(data)

# Create a workbook and add the dataframe to the sheet
wb = Workbook()
ws = wb.active

# Write the DataFrame to the worksheet
for r_idx, row in enumerate(dataframe_to_rows(df, index=False, header=True), 1):
    ws.append(row)

# Define the list of allowed values for the drop-down
drop_down_list = ['Male', 'Female', 'Other']

# Create a DataValidation object for the drop-down list
dv = DataValidation(type="list", formula1=f'"{",".join(drop_down_list)}"', showDropDown=False)

# Optional: Add an input prompt when the user selects the cell
dv.prompt = "Select Gender from the list"
dv.promptTitle = "Gender Selection"

# Apply the data validation to the "Gender" column (assuming it's column C)
ws.add_data_validation(dv)

# Extend the drop-down for all rows in the "Gender" column (starting from C2)
max_row = len(df) + 100  # Adjust the number of rows if needed
dv.add(f"C2:C{max_row}")  # Apply the drop-down from C2 to C{max_row}

# Save the workbook
output_path = 'output_with_dropdown_visible.xlsx'
wb.save(output_path)

print(f"Excel file saved with a drop-down in the 'Gender' column for all rows.")


Excel file saved with a drop-down in the 'Gender' column for all rows.


In [1]:
import pandas as pd

# Assuming your data is in a CSV file or an Excel file
# Replace 'your_file.xlsx' with the path to your Excel file if you're reading from one
# df = pd.read_excel('your_file.xlsx')

# Sample DataFrame to simulate the input
data = {
    'Business Framework': ['Global CMB', 'Other Framework','wpb'],  # Sample values
    'Country R3': ['UK', 'US','IN'],  # Sample values
}

# Create a DataFrame from the sample data
df = pd.DataFrame(data)

# Prepare an empty list to collect the expanded rows
expanded_rows = []

# Loop through each row in the original DataFrame
for index, row in df.iterrows():
    # Repeat each row 8 times with GCB values from 0 to 7
    for gcb_value in range(8):
        # Append the row values with the new GCB value to the list
        expanded_rows.append({
            'Business Framework': row['Business Framework'],
            'Country R3': row['Country R3'],
            'GCB': gcb_value
        })

# Create a new DataFrame from the expanded rows
expanded_df = pd.DataFrame(expanded_rows)

# Save the final DataFrame to a new Excel file or CSV
# expanded_df.to_excel('expanded_output.xlsx', index=False)
print(expanded_df)


   Business Framework Country R3  GCB
0          Global CMB         UK    0
1          Global CMB         UK    1
2          Global CMB         UK    2
3          Global CMB         UK    3
4          Global CMB         UK    4
5          Global CMB         UK    5
6          Global CMB         UK    6
7          Global CMB         UK    7
8     Other Framework         US    0
9     Other Framework         US    1
10    Other Framework         US    2
11    Other Framework         US    3
12    Other Framework         US    4
13    Other Framework         US    5
14    Other Framework         US    6
15    Other Framework         US    7
16                wpb         IN    0
17                wpb         IN    1
18                wpb         IN    2
19                wpb         IN    3
20                wpb         IN    4
21                wpb         IN    5
22                wpb         IN    6
23                wpb         IN    7


In [ ]:
from selenium import webdriver
import time

# Step 1: Set up Edge WebDriver using basic options
options = webdriver.EdgeOptions()

# Attempt to launch Edge without specifying the Service
driver = webdriver.Edge(options=options)

# Step 2: Open a blank tab to avoid the slow homepage
driver.execute_script("window.open('');")
time.sleep(1)  # Wait for the new tab to open

# Step 3: Switch to the new tab
driver.switch_to.window(driver.window_handles[1])

# Step 4: Navigate to the target URL in the new tab
target_url = 'https://your-target-website.com'  # Replace with your actual URL
driver.get(target_url)

# Step 5: Close the default homepage tab (which is the first tab)
driver.switch_to.window(driver.window_handles[0])
driver.close()

# Step 6: Switch back to the target website tab
driver.switch_to.window(driver.window_handles[0])

# Step 7: Let the browser stay open for some time
time.sleep(10)

# Finally, close the browser
driver.quit()


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Path to your Microsoft Edge WebDriver (Make sure it matches your browser version)
edge_driver_path = "path/to/msedgedriver.exe"

# Set up Edge WebDriver
service = Service(edge_driver_path)
driver = webdriver.Edge(service=service)

# Open the URL
driver.get('https://example.com')  # Replace with your actual URL

# Wait for the page to load and for the SSO button to be clickable
WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'My organization's single sign-on')]"))
)

# Locate and click the "SSO" button
sso_button = driver.find_element(By.XPATH, "//button[contains(text(), 'My organization's single sign-on')]")
sso_button.click()

# Let the SSO process complete
# You can wait for the page to load after the login
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.ID, "some_element_after_login"))  # Replace with an element that appears after login
)

# You can now proceed to interact with the logged-in page


In [ ]:
def options_to_dict(options):
    options_dict = {}
    for key in dir(options):
        if not key.startswith('__'):
            options_dict[key] = getattr(options, key)
    return options_dict

edge_options = Options()
edge_options.use_chromium = True

options_dict = options_to_dict(edge_options)
options_json = json.dumps(options_dict)

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Set up the browser and SSO settings
driver = webdriver.Chrome()  # Replace with your default browser

# Open the website and log in using default SSO
driver.get("https://example.com")  # Replace with the website URL

# Wait for the login process to complete
WebDriverWait(driver, 10).until(EC.title_contains("Logged in"))

# Navigate to the page with filters
driver.get("https://example.com/filters")  # Replace with the filter page URL

# Identify and interact with the filter elements
filter_elements = driver.find_elements_by_css_selector(".filter-option")
for element in filter_elements:
    # Perform the necessary actions to change the filter options
    element.click()  # Replace with the actual action required

# Click the ellipsis button to download the file
download_button = driver.find_element_by_css_selector(".download-button")
download_button.click()

# Specify the file path and name for the downloaded file
file_path = "/path/to/downloaded/file.csv"  # Replace with the desired file path and name

# Close the browser
driver.quit()

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import time

# Initialize the browser
driver = webdriver.Chrome(service=Service('/path/to/chromedriver'))

# Open the website
driver.get('https://example.com')  # Replace with the actual URL

# Wait for the page to load
time.sleep(3)

# Find the filter element by its label text (XPath)
filter_element = driver.find_element(By.XPATH, "//em[text()='BF Level 1 Name']")

# Click on the filter element
filter_element.click()

# Additional interaction (if needed)

# Close the browser after operations
driver.quit()


In [ ]:
import pandas as pd
from datetime import datetime

# Function to map 'Mxx' to actual periods
def map_month_to_period(month_str, year):
    months_map = {
        "M01": "Jan", "M02": "Feb", "M03": "Mar", "M04": "Apr", "M05": "May", 
        "M06": "Jun", "M07": "Jul", "M08": "Aug", "M09": "Sep", "M10": "Oct", 
        "M11": "Nov", "M12": "Dec"
    }
    month_abbr = months_map.get(month_str)
    return f"{month_abbr}-{year}"

# Process HC sheet and populate the 'Paste' sheet
def process_hc_sheet(df, paste_df, sheet_name, year):
    # Find the last 3 columns with 'Mxx' values for the months
    month_cols = [col for col in df.columns if 'M' in col]
    if len(month_cols) < 3:
        print(f"Not enough 'Mxx' columns found in {sheet_name}")
        return

    # Sort the columns to get the last three
    last_3_months = month_cols[-3:]
    
    for month_col in last_3_months:
        month_num = month_col[:3]  # Extract Mxx (e.g., M07)
        period = map_month_to_period(month_num, year)
        
        # Prepare the 'Paste' DataFrame row by row
        for _, row in df.iterrows():
            paste_row = {
                'File Name': sheet_name,
                'Entity': row['Entity'],
                'Function': row['RTN Level 4'],
                'FTE/Contractor': row[month_col],  # Value from the month column
                'Period': period,
                'Attribute Type': 'MoM',
            }
            # Append to 'Paste' DataFrame
            paste_df = paste_df.append(paste_row, ignore_index=True)
    
    return paste_df

# Main script to process the sheets
def consolidate_hc_sheets(sheets_dict, year):
    # Initialize the 'Paste' DataFrame
    paste_columns = [
        'File Name','Level 3','Level 4','Cost Grouping','Cost Type','Finance Region', 
        'Attribute Type','Period','Cost','FTE/Contractor','Country','Level 3.5','Level 4.5',
        'Entity','Function','Mapped Country','MICA'
    ]
    paste_df = pd.DataFrame(columns=paste_columns)

    # Iterate through the sheets
    for sheet_name, df in sheets_dict.items():
        if 'HC' in sheet_name:
            paste_df = process_hc_sheet(df, paste_df, sheet_name, year)

    return paste_df

# Example usage
# Assuming 'sheets_dict' contains all loaded sheets as DataFrames
sheets_dict = {
    'HC_Sheet1': pd.DataFrame(...),  # Replace with actual loaded sheet data
    'HC_Sheet2': pd.DataFrame(...),
    # Add more sheets as needed
}

# Call the function to process and consolidate HC sheets
final_paste_df = consolidate_hc_sheets(sheets_dict, 2024)
print(final_paste_df.head())


In [ ]:
import pandas as pd
from pyxlsb import open_workbook
from openpyxl import Workbook

# Path to the xlsb file (wb2)
wb2_path = 'path_to_wb2.xlsb'

# New Excel workbook to store the sheets that are not excluded
new_wb = Workbook()
new_wb.remove(new_wb.active)  # Remove the default sheet

# Function to find the header row based on 'Entity', 'RTN Level 4', 'Account'
def find_header_row(sheet):
    header_row = None
    # Iterate through rows to find the correct header row
    for row_num, row in enumerate(sheet.rows()):
        values = [item.v for item in row]
        if len(values) >= 3 and values[0] == 'Entity' and values[1] == 'RTN Level 4' and values[2] == 'Account':
            header_row = row_num  # Store the correct header row index (0-based)
            break
    return header_row

# Function to process a sheet and clean it up
def process_sheet(sheet, sheet_name):
    data = []
    header_row = find_header_row(sheet)

    if header_row is None:
        print(f"Header row not found in sheet: {sheet_name}")
        return None

    # Read the sheet into a list of lists (rows of data)
    for row in sheet.rows():
        data.append([item.v for item in row])

    # Convert data to a pandas dataframe
    df = pd.DataFrame(data)

    # Check the values of E15 and F15 (columns with index 4 and 5)
    try:
        E15_val = str(df.iloc[header_row + 1, 4])  # Value in column E (0-indexed, so 4)
        F15_val = str(df.iloc[header_row + 1, 5])  # Value in column F (0-indexed, so 5)
        print(f"Debug: E15 value: {E15_val}, F15 value: {F15_val}")
    except IndexError:
        E15_val = ''
        F15_val = ''
        print(f"Debug: E15 or F15 index out of range")

    # Define possible month headers
    month_headers = ['M{:02d}'.format(i) for i in range(1, 13)]  # Generates 'M01' to 'M12'
    print(f"Debug: Month headers: {month_headers}")

    # Prepare headers for columns A to C from row 15
    headers_A_to_C = df.iloc[header_row + 1, :3].values  # Columns A to C headers from row 15
    print(f"Debug: Headers A to C: {headers_A_to_C}")

    if E15_val in month_headers and F15_val in month_headers:
        # Use row 15 for headers for all columns
        headers_D_onwards = df.iloc[header_row + 1, 3:].values  # Columns D onwards headers from row 15
        headers = list(headers_A_to_C) + list(headers_D_onwards)
        start_data_row = header_row + 2  # Data starts after the header row
        print("Debug: Using headers from row 15 for columns D onwards")
    else:
        # Use row 14 for columns D onwards
        headers_D_onwards = df.iloc[header_row, 3:].values  # Columns D onwards headers from row 14
        headers = list(headers_A_to_C) + list(headers_D_onwards)
        start_data_row = header_row + 1  # Data starts after row 14
        print("Debug: Using headers from row 14 for columns D onwards")

    # Set the correct headers for the dataframe
    df = df.iloc[start_data_row:].reset_index(drop=True)  # Keep only rows after the header
    df.columns = headers

    # Drop rows where all values are NaN or None
    df.dropna(how='all', inplace=True)

    return df

# Open the xlsb workbook and process valid sheets
with open_workbook(wb2_path) as wb2:
    for sheetname in wb2.sheets:
        # Exclude sheets with 'excl', 'Pivot', or 'TM1'
        if any(exclude_word.lower() in sheetname.lower() for exclude_word in ['excl', 'pivot', 'tm1']):
            print(f"Skipping sheet: {sheetname}")
            continue

        with wb2.get_sheet(sheetname) as sheet:
            print(f"Processing sheet: {sheetname}")
            df = process_sheet(sheet, sheetname)

            if df is not None:
                # Save DataFrame to a new sheet in the new workbook
                new_sheet = new_wb.create_sheet(title=sheetname[:31])  # Sheet names max length is 31 characters

                # Write the headers to the sheet
                new_sheet.append(df.columns.tolist())

                # Write the data rows to the sheet
                for row in df.itertuples(index=False, name=None):
                    new_sheet.append(row)

# Save the new workbook with the non-excluded sheets
output_path = 'non_excluded_sheets_with_headers.xlsx'
new_wb.save(output_path)
print(f"Workbook with non-excluded sheets and correct headers has been saved to '{output_path}'")


In [ ]:
import pandas as pd
from pyxlsb import open_workbook
from openpyxl import Workbook

# Path to the xlsb file (wb2)
wb2_path = 'path_to_wb2.xlsb'

# New Excel workbook to store the results
new_wb = Workbook()
new_wb.remove(new_wb.active)  # Remove the default sheet

# Function to find the header row based on 'Entity', 'RTN Level 4', 'Account'
def find_header_row(sheet):
    header_row = None
    # Iterate through rows to find the correct header row
    for row_num, row in enumerate(sheet.rows()):
        values = [item.v for item in row]
        if len(values) >= 3 and values[0] == 'Entity' and values[1] == 'RTN Level 4' and values[2] == 'Account':
            header_row = row_num  # Store the correct header row index (0-based)
            break
    return header_row

# Function to process a sheet and clean it up
def process_sheet(sheet, sheet_name):
    data = []
    header_row = find_header_row(sheet)

    if header_row is None:
        print(f"Header row not found in sheet: {sheet_name}")
        return None

    # Read the sheet into a list of lists (rows of data)
    for row in sheet.rows():
        data.append([item.v for item in row])

    # Convert data to a pandas dataframe
    df = pd.DataFrame(data)

    # Check the values of E15 and F15 (row 15 is 0-based index 14 in pandas)
    E15_val = str(df.iloc[header_row + 1, 4])  # Value in column E (0-indexed, so 4)
    F15_val = str(df.iloc[header_row + 1, 5])  # Value in column F (0-indexed, so 5)

    # Define possible month headers
    month_headers = ['M01', 'M02', 'M03', 'M04', 'M05', 'M06', 'M07', 'M08', 'M09', 'M10', 'M11', 'M12']

    # Adjust headers for columns D onwards based on the content of columns E and F
    if E15_val in month_headers and F15_val in month_headers:
        # Use row 15 for headers (index 14 in pandas)
        headers = df.iloc[header_row + 1].values  # Take headers from row 15
        start_data_row = header_row + 2  # Data starts after row 15
    else:
        # Use row 14 for headers (index 13 in pandas)
        headers = df.iloc[header_row].values  # Take headers from row 14
        start_data_row = header_row + 1  # Data starts after row 14

    # Set the correct headers for the dataframe
    df.columns = headers
    df = df[start_data_row:].reset_index(drop=True)  # Keep only rows after the header

    # Drop rows where all values are NaN or None
    df.dropna(how='all', inplace=True)
    
    return df

# Open the xlsb workbook and process valid sheets
with open_workbook(wb2_path) as wb2:
    for sheetname in wb2.sheets:
        # Exclude sheets with 'excl', 'Pivot', or 'TM1'
        if any(exclude_word.lower() in sheetname.lower() for exclude_word in ['excl', 'Pivot', 'TM1']):
            print(f"Skipping sheet: {sheetname}")
            continue

        with wb2.get_sheet(sheetname) as sheet:
            print(f"Processing sheet: {sheetname}")
            df = process_sheet(sheet, sheetname)
            
            if df is not None:
                # Save DataFrame to a new sheet in the new workbook
                new_sheet = new_wb.create_sheet(title=sheetname)
                for r in df.itertuples(index=False, name=None):
                    new_sheet.append(r)

# Save the new workbook with the updated data
output_path = 'updated_wb.xlsx'
new_wb.save(output_path)
print(f"Data has been written to '{output_path}'")


In [7]:
import tkinter as tk
from tkinter import filedialog, messagebox

class ExcelSheetSelector:
    def __init__(self, root):
        self.root = root
        self.root.title("Excel Sheet Selector")

        # Custom labels for each file selection
        self.custom_labels = [
            "Select OFile:", "Select CFile:", "Select DFile:", "Select Fourth File:",
            "Select Fifth File:", "Select Sixth File:", "Select Seventh File:", "Select Eighth File:"
        ]

        # List to hold selected file paths, initially None
        self.selected_files = [None] * 8
        self.labels = []

        # Create custom labels and buttons for selecting files
        for i in range(8):
            # Create a custom label for each file selection
            custom_label = tk.Label(root, text=self.custom_labels[i], width=20, anchor='w')
            custom_label.grid(row=i, column=0, padx=10, pady=5)

            # Create a label to display the selected file path
            label = tk.Label(root, text="Not selected", width=60, anchor='w')
            label.grid(row=i, column=1, padx=10, pady=5)
            self.labels.append(label)

            # Create a button for each file selection
            button = tk.Button(root, text="Browse", command=lambda index=i: self.select_file(index))
            button.grid(row=i, column=2, padx=10, pady=5)

        # Button to confirm file selections
        confirm_button = tk.Button(root, text="Confirm Selection", command=self.confirm_selection)
        confirm_button.grid(row=8, column=0, columnspan=3, pady=20)

        # Predefined variable names for file paths
        self.OFile = None
        self.CFile = None
        self.DFile = None
        self.file4 = None
        self.file5 = None
        self.file6 = None
        self.file7 = None
        self.file8 = None

    def select_file(self, index):
        # Open file dialog to select a file
        file_path = filedialog.askopenfilename(title="Select a File", filetypes=[("All Files", "*.*")])
        
        if file_path:  # If a file is selected
            self.selected_files[index] = file_path  # Save the selected file
            self.labels[index].config(text=file_path)  # Update label with the file path

    def confirm_selection(self):
        # Check how many files have been selected
        num_selected = sum(1 for file in self.selected_files if file)

        if num_selected == 0:
            messagebox.showwarning("Warning", "No files selected.")
        else:
            # Show confirmation dialog with options to Confirm or Select More Files
            result = messagebox.askquestion(
                "Confirmation", 
                f"You have selected {num_selected} files. Do you want to proceed?", 
                icon='question'
            )
            
            if result == 'yes':
                # Directly assign file paths to predefined variables
                self.assign_file_paths()
                messagebox.showinfo("Proceed", "Files are selected. Proceeding with the application.")
                # Print assigned files for verification
                print(self.OFile, self.CFile, self.DFile, self.file4, self.file5, self.file6, self.file7, self.file8)
                # Close the GUI
                self.root.destroy()
            else:
                # Allow the user to select more files
                messagebox.showinfo("Select More", "Please select more files.")

    def assign_file_paths(self):
        """Assigns selected file paths to predefined variables."""
        self.OFile = self.selected_files[0] if len(self.selected_files) > 0 else None
        self.CFile = self.selected_files[1] if len(self.selected_files) > 1 else None
        self.DFile = self.selected_files[2] if len(self.selected_files) > 2 else None
        self.file4 = self.selected_files[3] if len(self.selected_files) > 3 else None
        self.file5 = self.selected_files[4] if len(self.selected_files) > 4 else None
        self.file6 = self.selected_files[5] if len(self.selected_files) > 5 else None
        self.file7 = self.selected_files[6] if len(self.selected_files) > 6 else None
        self.file8 = self.selected_files[7] if len(self.selected_files) > 7 else None

# Initialize the Tkinter window and ExcelSheetSelector class
if __name__ == "__main__":
    root = tk.Tk()
    app = ExcelSheetSelector(root)
    root.mainloop()


C:/Users/KS/data.xlsx None None None None None C:/Users/KS/final_output.xlsx None


In [5]:
import tkinter as tk
from tkinter import filedialog, messagebox

def select_file(index):
    # Open file dialog to select a file
    file_path = filedialog.askopenfilename(title="Select a File", filetypes=[("All Files", "*.*")])
    
    if file_path:  # If a file is selected
        selected_files[index] = file_path  # Save the selected file
        labels[index].config(text=file_path)  # Update label with the file path

def confirm_selection():
    # Check how many files have been selected
    num_selected = sum(1 for file in selected_files if file)
    
    if num_selected == 0:
        messagebox.showwarning("Warning", "No files selected.")
    else:
        # Show confirmation dialog with options to Confirm or Select More Files
        result = messagebox.askquestion("Confirmation", f"You have selected {num_selected} files. Do you want to proceed?", icon='question')
        
        if result == 'yes':
            # Proceed and close the application
            messagebox.showinfo("Proceed", "Files are selected. Proceeding with the application.")
            root.destroy()
        else:
            # Allow the user to select more files
            messagebox.showinfo("Select More", "Please select more files.")

# Initialize the Tkinter window
root = tk.Tk()
root.title("File Selector")

# List to hold selected file paths
selected_files = [None] * 8
labels = []

# Create labels and buttons for selecting files
for i in range(8):
    # Create a label for each file selection
    label = tk.Label(root, text=f"File {i+1}: Not selected", width=100, anchor='w')
    label.grid(row=i, column=0, padx=10, pady=5)
    labels.append(label)
    
    # Create a button for each file selection
    button = tk.Button(root, text="Select File", command=lambda index=i: select_file(index))
    button.grid(row=i, column=1, padx=10, pady=5)

# Button to confirm file selections
confirm_button = tk.Button(root, text="Confirm Selection", command=confirm_selection)
confirm_button.grid(row=8, column=0, columnspan=2, pady=20)

root.mainloop()


In [ ]:
import tkinter as tk
from tkinter import filedialog, messagebox

def select_files():
    # Open the file dialog to select files
    file_paths = filedialog.askopenfilenames(title="Select Files", filetypes=[("All Files", "*.*")])
    
    # Update the listbox with selected files
    listbox.delete(0, tk.END)
    for file in file_paths:
        listbox.insert(tk.END, file)
    
    # Save the selected files to the global variable
    selected_files.clear()
    selected_files.extend(file_paths)
    
    # Confirm the number of selected files
    if len(selected_files) == 0:
        messagebox.showinfo("Information", "No files selected.")
    else:
        messagebox.showinfo("Information", f"You have selected {len(selected_files)} files.")

def confirm_selection():
    # Proceed with the application logic
    if len(selected_files) > 0:
        messagebox.showinfo("Proceed", "Files are selected. Proceeding with the application.")
    else:
        messagebox.showwarning("Warning", "No files selected. Proceeding with the application.")

# Initialize the Tkinter window
root = tk.Tk()
root.title("File Selector")

selected_files = []

# Create a Listbox to display selected files
listbox = tk.Listbox(root, width=100, height=10)
listbox.pack(pady=10)

# Add buttons to select files and confirm selection
select_button = tk.Button(root, text="Select Files", command=select_files)
select_button.pack(pady=5)

confirm_button = tk.Button(root, text="Confirm Selection", command=confirm_selection)
confirm_button.pack(pady=5)

root.mainloop()


In [1]:
import pandas as pd

# Sample DataFrame
data = {
    'Record Type': ['EUC', 'EUC', 'EUC', 'Other', 'EUC'],
    'Record ID': [1, 2, 3, 4, 5],
    'Record Name': ['Name1', 'Name2', 'Name3', 'Name4', 'Name5'],
    'Staff ID': [101, 102, 102, 102, 103],
    'Position Changed': ['Left', 'No Change', 'Yes', 'Left', 'No Change']
}
df = pd.DataFrame(data)

# Define a function to concatenate control items and determine control domain
def concatenate_control_items(group):
    control_items = '\n'.join(group['Record Type'] + ' ' + group['Record ID'].astype(str) + ' ' + group['Record Name'])
    record_types = group['Record Type'].unique()
    if len(record_types) > 1:
        control_domain = 'mix'
    elif 'EUC' in record_types:
        control_domain = 'EUC only'
    else:
        control_domain = 'Other'
    return pd.Series({
        'Control Items': control_items,
        'Control Domain': control_domain
    })

# Group by 'Staff ID' and apply the function
result_df = df.groupby('Staff ID').apply(concatenate_control_items).reset_index()

# Merge the result back to the original DataFrame to maintain original rows
final_df = df.drop(columns=['Record Type', 'Record ID', 'Record Name']).drop_duplicates().merge(result_df, on='Staff ID', how='left')

# Print the final DataFrame
print(final_df)

# Save the result to an Excel file if needed
final_df.to_excel('final_output.xlsx', index=False)


   Staff ID Position Changed                            Control Items  \
0       101             Left                              EUC 1 Name1   
1       102        No Change  EUC 2 Name2\nEUC 3 Name3\nOther 4 Name4   
2       102              Yes  EUC 2 Name2\nEUC 3 Name3\nOther 4 Name4   
3       102             Left  EUC 2 Name2\nEUC 3 Name3\nOther 4 Name4   
4       103        No Change                              EUC 5 Name5   

  Control Domain  
0       EUC only  
1            mix  
2            mix  
3            mix  
4       EUC only  


In [6]:
import tkinter as tk
from tkinter import filedialog, messagebox

class ExcelSheetSelector:
    def __init__(self, root):
        self.root = root
        self.root.title("Excel Sheet Selector")

        self.file_paths = [None] * 7
        self.labels = []

        # Specific labels for each file selection
        self.label_texts = [
            "Select Humans file:",
            "Select Animals file:",
            "Select Plants file:",
            "Select Minerals file:",
            "Select Microbes file:",
            "Select Fungi file:",
            "Select Algae file:"
        ]

        # Create labels and buttons for 7 Excel sheets
        for i in range(7):
            label = tk.Label(root, text=self.label_texts[i])
            label.grid(row=i, column=0, padx=10, pady=5)
            
            button = tk.Button(root, text="Browse", command=lambda i=i: self.select_file(i))
            button.grid(row=i, column=1, padx=10, pady=5)
            
            file_label = tk.Label(root, text="No file selected")
            file_label.grid(row=i, column=2, padx=10, pady=5)
            self.labels.append(file_label)
        
        # Button to check and print selected file paths
        self.print_button = tk.Button(root, text="Confirm Selections", command=self.check_and_confirm)
        self.print_button.grid(row=7, column=0, columnspan=3, pady=10)

    def select_file(self, index):
        self.root.withdraw()  # Hide the main window
        file_path = filedialog.askopenfilename(filetypes=[("Excel files", "*.xlsx *.xls")])
        self.root.deiconify()  # Show the main window again
        if file_path:
            self.file_paths[index] = file_path
            self.labels[index].config(text=file_path)

    def check_and_confirm(self):
        if all(self.file_paths):
            self.root.destroy()  # Close the window gracefully
        else:
            messagebox.showwarning("Warning", "Please select all files before proceeding.")

# Create the main window
root = tk.Tk()
app = ExcelSheetSelector(root)
root.mainloop()

# After closing the window, capture the file paths into separate variables
humans_file, animals_file, plants_file, minerals_file, microbes_file, fungi_file, algae_file = app.file_paths

# Print the captured file paths (for verification)
print(f"Humans file: {humans_file}")
print(f"Animals file: {animals_file}")
print(f"Plants file: {plants_file}")
print(f"Minerals file: {minerals_file}")
print(f"Microbes file: {microbes_file}")
print(f"Fungi file: {fungi_file}")
print(f"Algae file: {algae_file}")


Humans file: C:/Users/KS/Start-Data-Analysis.xlsx
Animals file: C:/Users/KS/Source.xlsx
Plants file: C:/Users/KS/sample.xlsx
Minerals file: C:/Users/KS/sam.xlsx
Microbes file: C:/Users/KS/sf2_output.xlsx
Fungi file: C:/Users/KS/modified_workbook1.xlsx
Algae file: C:/Users/KS/modified_workbook.xlsx


In [1]:
import pandas as pd

# Sample data creation for demonstration
data = {
    'Record Type': ['Issue', 'Issue', 'Action', 'Issue', 'Action'],
    'Record ID': [101, 102, 103, 104, 105],
    'Record Name': ['A', 'B', 'C', 'D', 'E'],
    'Record Details': ['Detail1', 'Detail2', 'Detail3', 'Detail4', 'Detail5'],
    'Staff ID': [1, 1, 2, 1, 2],
    'Position Changed': ['Yes', 'No', 'Yes', 'No', 'Yes']
}

# Create DataFrame
df = pd.DataFrame(data)

# Define a function to concatenate the desired fields
def concatenate_records(group):
    return '\n'.join(group['Record Type'] + ' ' + group['Record ID'].astype(str) + ' ' + group['Record Name'])

# Group by 'Staff ID' and apply the concatenation function
grouped_df = df.groupby('Staff ID').apply(lambda x: pd.Series({
    'Control Items': concatenate_records(x),
    'Position Changed': x['Position Changed'].iloc[0]  # Assuming you want to keep the first 'Position Changed' value
})).reset_index()

# Optional: Include any other columns you want to keep from the original DataFrame
# For demonstration, I'm adding 'Record Details' concatenated in the same way
grouped_df['Record Details'] = df.groupby('Staff ID')['Record Details'].apply(lambda x: '\n'.join(x)).reset_index(drop=True)

# Display the final DataFrame
print(grouped_df)

# Save the final DataFrame to an Excel file
# grouped_df.to_excel('path_to_output_file.xlsx', index=False)


   Staff ID                          Control Items Position Changed  \
0         1  Issue 101 A\nIssue 102 B\nIssue 104 D              Yes   
1         2             Action 103 C\nAction 105 E              Yes   

              Record Details  
0  Detail1\nDetail2\nDetail4  
1           Detail3\nDetail5  


In [ ]:
import pandas as pd

def lookup_and_merge(file1_path, file2_path, vOld_path):
    # Read the first file into a DataFrame
    df1 = pd.read_excel(file1_path)
    
    # Read the second file into a DataFrame
    df2 = pd.read_excel(file2_path)
    
    # Read the vOld file into a DataFrame
    vOld = pd.read_excel(vOld_path)
    
    # Ensure the Staff ID in both DataFrames are treated as numeric for accurate matching
    df1['Employee ID'] = pd.to_numeric(df1['Employee ID'], errors='coerce').astype('Int64')
    df2['Staff ID'] = pd.to_numeric(df2['Staff ID'], errors='coerce').astype('Int64')
    vOld['Staff ID'] = pd.to_numeric(vOld['Staff ID'], errors='coerce').astype('Int64')
    
    # Perform the lookup by merging the DataFrames on Staff ID and Employee ID
    merged_df = df2.merge(df1[['Employee ID', 'Position Changed']], 
                          left_on='Staff ID', right_on='Employee ID', 
                          how='left')
    
    # Drop the redundant 'Employee ID' column from the merged DataFrame
    merged_df.drop(columns=['Employee ID'], inplace=True)
    
    # Filter the DataFrame to retain only rows where 'Position Changed' is 'Left' or 'Yes'
    filtered_df = merged_df[merged_df['Position Changed'].isin(['Left', 'Yes'])]
    
    # Ensure 'Staff ID' and 'Position Changed' columns are filled properly
    filtered_df['Staff ID'] = filtered_df['Staff ID'].fillna('No ID')
    filtered_df['Position Changed'] = filtered_df['Position Changed'].fillna('No Change')
    
    # Perform the second lookup by merging the filtered_df with vOld on Staff ID
    final_df = filtered_df.merge(vOld[['Staff ID', 'Employee Name', 'Employee Business Email Address', 'Global Career Band']], 
                                 on='Staff ID', 
                                 how='left')
    
    # Rename the fetched columns
    final_df.rename(columns={
        'Employee Name': 'Functional Manager Employee Name',
        'Employee Business Email Address': 'Functional Manager Email',
        'Global Career Band': 'Functional Manager GCB'
    }, inplace=True)
    
    # Return the final DataFrame
    return final_df

# Paths to the Excel files
file1_path = 'path_to_file1.xlsx'  # File with Employee ID, Position Changed, etc.
file2_path = 'path_to_file2.xlsx'  # File with Record ID, Record Name, Record Details, Staff ID
vOld_path = 'path_to_vOld.xlsx'    # File with additional details based on Staff ID

# Perform the lookup, merge, and filter
final_df = lookup_and_merge(file1_path, file2_path, vOld_path)

# Display the final DataFrame
print(final_df)

# Save the final DataFrame to a new Excel file
final_df.to_excel('path_to_output_file.xlsx', index=False)


In [ ]:
import pandas as pd

def lookup_and_merge(file1_path, file2_path):
    # Read the first file into a DataFrame
    df1 = pd.read_excel(file1_path)
    
    # Read the second file into a DataFrame
    df2 = pd.read_excel(file2_path)
    
    # Ensure the Staff ID in both DataFrames are strings for accurate matching
    df1['Employee ID'] = df1['Employee ID'].astype(str)
    df2['Staff ID'] = df2['Staff ID'].astype(str)
    
    # Perform the lookup by merging the DataFrames on Staff ID and Employee ID
    merged_df = df2.merge(df1[['Employee ID', 'Position Changed']], 
                          left_on='Staff ID', right_on='Employee ID', 
                          how='left')
    
    # Drop the redundant 'Employee ID' column from the merged DataFrame
    merged_df.drop(columns=['Employee ID'], inplace=True)
    
    # Return the merged DataFrame
    return merged_df

# Paths to the Excel files
file1_path = 'path_to_file1.xlsx'  # File with Employee ID, Position Changed, etc.
file2_path = 'path_to_file2.xlsx'  # File with Record ID, Record Name, Record Details, Staff ID

# Perform the lookup and merge
final_df = lookup_and_merge(file1_path, file2_path)

# Display the final DataFrame
print(final_df)

# If needed, save the final DataFrame to a new Excel file
# final_df.to_excel('path_to_output_file.xlsx', index=False)


In [ ]:
import pandas as pd

def process_files(vnew_path, vold_path):
    # Read the Excel files into DataFrames
    vNew = pd.read_excel(vnew_path)
    vOld = pd.read_excel(vold_path)
    
    # Check and print initial dtypes of columns
    print("Initial dtypes of vNew:")
    print(vNew.dtypes)
    print("\nInitial dtypes of vOld:")
    print(vOld.dtypes)
    
    # Function to convert columns to int, skipping invalid rows
    def convert_to_int(df, column):
        df[column] = pd.to_numeric(df[column], errors='coerce')
        df.dropna(subset=[column], inplace=True)
        df[column] = df[column].astype(int)
        return df

    # Convert 'Employee ID' and 'Position Number' to integers
    vNew = convert_to_int(vNew, 'Employee ID')
    vNew = convert_to_int(vNew, 'Position Number')
    vOld = convert_to_int(vOld, 'Employee ID')
    vOld = convert_to_int(vOld, 'Position Number')
    
    # Check and print dtypes of columns after coercion attempt
    print("\nDtypes of vNew after coercion attempt:")
    print(vNew.dtypes)
    print("\nDtypes of vOld after coercion attempt:")
    print(vOld.dtypes)
    
    # Select necessary columns from vNew
    vNew_selected = vNew[['Employee ID', 'Position Number', 'BF Level 4']].copy()
    vNew_selected.rename(columns={
        'Position Number': 'new Position Number',
        'BF Level 4': 'new BF Level 4'
    }, inplace=True)
    
    # Select necessary columns from vOld
    vOld_selected = vOld[['Employee ID', 'Position Number', 'BF Level 4']].copy()
    vOld_selected.rename(columns={
        'Position Number': 'old Position Number',
        'BF Level 4': 'old BF Level 4'
    }, inplace=True)
    
    # Merge the DataFrames on Employee ID using left join
    merged_df = pd.merge(vOld_selected, vNew_selected, on='Employee ID', how='left')
    
    # Create the Position Changed column based on the specified conditions
    def determine_position_changed(row):
        if pd.isna(row['new Position Number']) and pd.isna(row['new BF Level 4']):
            return 'Left'
        elif (row['old Position Number'] != row['new Position Number']) or (row['old BF Level 4'] != row['new BF Level 4']):
            return 'Yes'
        else:
            return 'No Change'
    
    merged_df['Position Changed'] = merged_df.apply(determine_position_changed, axis=1)
    
    return merged_df

# Example usage
vnew_path = 'path_to_vNew.xlsx'
vold_path = 'path_to_vOld.xlsx'

final_df = process_files(vnew_path, vold_path)

# Display the final DataFrame
print(final_df)


In [ ]:
import pandas as pd

def process_files(vnew_path, vold_path):
    # Read the Excel files into DataFrames
    vNew = pd.read_excel(vnew_path)
    vOld = pd.read_excel(vold_path)
    
    # Select necessary columns from vNew
    vNew_selected = vNew[['Employee ID', 'Position Number', 'BF Level 4']].copy()
    vNew_selected.rename(columns={
        'Position Number': 'new Position Number',
        'BF Level 4': 'new BF Level 4'
    }, inplace=True)
    
    # Select necessary columns from vOld
    vOld_selected = vOld[['Employee ID', 'Position Number', 'BF Level 4']].copy()
    vOld_selected.rename(columns={
        'Position Number': 'old Position Number',
        'BF Level 4': 'old BF Level 4'
    }, inplace=True)
    
    # Merge the DataFrames on Employee ID using left join
    merged_df = pd.merge(vOld_selected, vNew_selected, on='Employee ID', how='left')
    
    # Create the Position Changed column based on the specified conditions
    def determine_position_changed(row):
        if pd.isna(row['new Position Number']) and pd.isna(row['new BF Level 4']):
            return 'Left'
        elif (row['old Position Number'] != row['new Position Number']) or (row['old BF Level 4'] != row['new BF Level 4']):
            return 'Yes'
        else:
            return 'No Change'
    
    merged_df['Position Changed'] = merged_df.apply(determine_position_changed, axis=1)
    
    return merged_df

# Example usage
vnew_path = 'path_to_vNew.xlsx'
vold_path = 'path_to_vOld.xlsx'

final_df = process_files(vnew_path, vold_path)

# Display the final DataFrame
print(final_df)


In [ ]:
import pandas as pd

def process_files(vnew_path, vold_path):
    # Read the Excel files into DataFrames
    vNew = pd.read_excel(vnew_path)
    vOld = pd.read_excel(vold_path)
    
    # Select necessary columns from vNew
    vNew_selected = vNew[['Employee ID', 'Position Number', 'BF Level 4']].copy()
    vNew_selected.rename(columns={
        'Position Number': 'new Position Number',
        'BF Level 4': 'new BF Level 4'
    }, inplace=True)
    
    # Select necessary columns from vOld
    vOld_selected = vOld[['Employee ID', 'Position Number', 'BF Level 4']].copy()
    vOld_selected.rename(columns={
        'Position Number': 'old Position Number',
        'BF Level 4': 'old BF Level 4'
    }, inplace=True)
    
    # Merge the DataFrames on Employee ID
    merged_df = pd.merge(vOld_selected, vNew_selected, on='Employee ID', how='outer')
    
    # Create the Position Changed column based on the specified conditions
    def determine_position_changed(row):
        if pd.isna(row['new Position Number']) and pd.isna(row['new BF Level 4']):
            return 'Left'
        elif (row['old Position Number'] != row['new Position Number']) or (row['old BF Level 4'] != row['new BF Level 4']):
            return 'Yes'
        else:
            return 'No Change'
    
    merged_df['Position Changed'] = merged_df.apply(determine_position_changed, axis=1)
    
    return merged_df

# Example usage
vnew_path = 'path_to_vNew.xlsx'
vold_path = 'path_to_vOld.xlsx'

final_df = process_files(vnew_path, vold_path)

# Display the final DataFrame
print(final_df)


In [ ]:
import pandas as pd
import re

def extract_staff_id(action_owner_col):
    """Extract Staff ID from the given column using regex."""
    if isinstance(action_owner_col, str):
        match = re.search(r'\((\d{8})\)', action_owner_col)
        return match.group(1) if match else None
    return None

def process_single_file(file_path, staff_id_col, issue_status_col, filter_condition,
                        record_id_col, record_name_col, record_details_col, record_type_value):
    """
    Process a single file to extract relevant data and transform it into a standardized format.
    
    Parameters:
        file_path (str): Path to the input Excel file.
        staff_id_col (str): Column name for the Staff ID extraction.
        issue_status_col (str): Column name for filtering records.
        filter_condition (str): Value to filter the records (e.g., 'open').
        record_id_col (str): Column name for 'Record ID'.
        record_name_col (str): Column name for 'Record Name'.
        record_details_col (str): Column name for 'Record Details'.
        record_type_value (str): Value to be placed in 'Record Type' column.
    
    Returns:
        pd.DataFrame: Processed DataFrame with standardized columns.
    """
    # Load the Excel file into a DataFrame
    df = pd.read_excel(file_path)
    
    # Ensure the staff ID column is treated as string
    df[staff_id_col] = df[staff_id_col].astype(str)
    
    # Filter the DataFrame based on the provided condition
    filtered_df = df[df[issue_status_col] == filter_condition]
    
    # Extract Staff ID from the specified column
    filtered_df['Staff ID'] = filtered_df[staff_id_col].apply(extract_staff_id)
    
    # Add the 'Record Type' column with the specified value
    filtered_df['Record Type'] = record_type_value
    
    # Create the final DataFrame with the required columns
    result_df = filtered_df[[record_id_col, record_name_col, record_details_col]].copy()
    result_df.rename(columns={
        record_id_col: 'Record ID',
        record_name_col: 'Record Name',
        record_details_col: 'Record Details'
    }, inplace=True)
    
    # Add 'Staff ID' and 'Record Type' columns
    result_df['Staff ID'] = filtered_df['Staff ID']
    result_df['Record Type'] = filtered_df['Record Type']
    
    # Return the result DataFrame
    return result_df

def concatenate_files(file_params_list):
    """
    Process multiple files and concatenate the results into a single DataFrame.
    
    Parameters:
        file_params_list (list of dict): List of dictionaries with parameters for each file.
    
    Returns:
        pd.DataFrame: Concatenated DataFrame with results from all files.
    """
    all_dfs = []
    
    for params in file_params_list:
        df = process_single_file(
            file_path=params['file_path'],
            staff_id_col=params['staff_id_col'],
            issue_status_col=params['issue_status_col'],
            filter_condition=params['filter_condition'],
            record_id_col=params['record_id_col'],
            record_name_col=params['record_name_col'],
            record_details_col=params['record_details_col'],
            record_type_value=params['record_type_value']
        )
        all_dfs.append(df)
    
    # Concatenate all DataFrames
    final_df = pd.concat(all_dfs, ignore_index=True)
    
    return final_df

# Example usage
file_params_list = [
    {
        'file_path': 'path_to_file1.xlsx',
        'staff_id_col': 'Action Owner',
        'issue_status_col': 'Issue Status',
        'filter_condition': 'open',
        'record_id_col': 'Issue ID',
        'record_name_col': 'Issue Title',
        'record_details_col': 'Issue Description',
        'record_type_value': 'Issue'
    },
    {
        'file_path': 'path_to_file2.xlsx',
        'staff_id_col': 'Owner',
        'issue_status_col': 'Status',
        'filter_condition': 'open',
        'record_id_col': 'ID',
        'record_name_col': 'Title',
        'record_details_col': 'Description',
        'record_type_value': 'Action Owner'
    },
    # Add parameters for other files here
]

# Process multiple files and get the final DataFrame
final_df = concatenate_files(file_params_list)

# Display the final DataFrame
print(final_df)


In [ ]:
import pandas as pd
import re

def extract_staff_id(action_owner_col):
    """Extract Staff ID from the given column using regex."""
    if isinstance(action_owner_col, str):
        match = re.search(r'\((\d{8})\)', action_owner_col)
        return match.group(1) if match else None
    return None

def process_file(file_path, staff_id_col, issue_status_col, filter_condition,
                 record_id_col, record_name_col, record_details_col):
    """
    Process a single file to extract relevant data and transform it into a standardized format.
    
    Parameters:
        file_path (str): Path to the input Excel file.
        staff_id_col (str): Column name for the Staff ID extraction.
        issue_status_col (str): Column name for filtering records.
        filter_condition (str): Value to filter the records (e.g., 'open').
        record_id_col (str): Column name for 'Record ID'.
        record_name_col (str): Column name for 'Record Name'.
        record_details_col (str): Column name for 'Record Details'.
    
    Returns:
        pd.DataFrame: Processed DataFrame with standardized columns.
    """
    # Load the Excel file into a DataFrame
    df = pd.read_excel(file_path)
    
    # Ensure the staff ID column is treated as string
    df[staff_id_col] = df[staff_id_col].astype(str)
    
    # Filter the DataFrame based on the provided condition
    filtered_df = df[df[issue_status_col] == filter_condition]
    
    # Extract Staff ID from the specified column
    filtered_df['Staff ID'] = filtered_df[staff_id_col].apply(extract_staff_id)
    
    # Add the 'Record Type' column with the value 'Issue'
    filtered_df['Record Type'] = 'Issue'
    
    # Create the final DataFrame with the required columns
    result_df = filtered_df[[staff_id_col, record_id_col, record_name_col, record_details_col]].copy()
    result_df.rename(columns={
        staff_id_col: 'Staff ID',
        record_id_col: 'Record ID',
        record_name_col: 'Record Name',
        record_details_col: 'Record Details'
    }, inplace=True)
    
    # Add the 'Record Type' column
    result_df['Record Type'] = 'Issue'
    
    # Return the result DataFrame
    return result_df

def process_multiple_files(file_paths, column_mapping, filter_condition):
    """
    Process multiple files and concatenate the results into a single DataFrame.
    
    Parameters:
        file_paths (list of str): List of paths to input Excel files.
        column_mapping (list of dict): List of dictionaries with column mappings for each file.
        filter_condition (str): Value to filter records (e.g., 'open').
    
    Returns:
        pd.DataFrame: Concatenated DataFrame with results from all files.
    """
    all_dfs = []
    
    for file_path, columns in zip(file_paths, column_mapping):
        df = process_file(
            file_path,
            staff_id_col=columns['staff_id'],
            issue_status_col=columns['issue_status'],
            filter_condition=filter_condition,
            record_id_col=columns['record_id'],
            record_name_col=columns['record_name'],
            record_details_col=columns['record_details']
        )
        all_dfs.append(df)
    
    # Concatenate all DataFrames
    final_df = pd.concat(all_dfs, ignore_index=True)
    
    return final_df

# Example usage
file_paths = [
    'path_to_file1.xlsx',
    'path_to_file2.xlsx',
    'path_to_file3.xlsx',
    'path_to_file4.xlsx',
    'path_to_file5.xlsx'
]

column_mapping = [
    {
        'staff_id': 'Action Owner',  # Column name to extract Staff ID
        'issue_status': 'Issue Status',  # Column name for issue status
        'record_id': 'Issue ID',  # Column name for Record ID
        'record_name': 'Issue Title',  # Column name for Record Name
        'record_details': 'Issue Description'  # Column name for Record Details
    },
    # Add mappings for other files here
]

filter_condition = 'open'  # Condition to filter records

# Process multiple files and get the final DataFrame
final_df = process_multiple_files(file_paths, column_mapping, filter_condition)

# Display the final DataFrame
print(final_df)


In [ ]:
import pandas as pd
import re

# Function to extract Staff ID using regular expression
def extract_staff_id(action_owner):
    if isinstance(action_owner, str):
        match = re.search(r'\((\d{8})\)', action_owner)
        return match.group(1) if match else None
    return None

# Load the Excel file into a DataFrame
df = pd.read_excel('path_to_your_file.xlsx')

# Ensure "Action Owner" column is treated as string
df['Action Owner'] = df['Action Owner'].astype(str)

# Filter the DataFrame for rows where 'Issue Status' is 'open'
open_issues_df = df[df['Issue Status'] == 'open']

# Extract Staff ID from "Action Owner" column for filtered rows
open_issues_df['Staff ID'] = open_issues_df['Action Owner'].apply(extract_staff_id)

# Add the 'Record Type' column with value 'Issue'
open_issues_df['Record Type'] = 'Issue'

# Create the final DataFrame with the required columns
result_df = open_issues_df[['Staff ID', 'Record Type', 'Issue ID', 'Issue Title', 'Issue Description']]
result_df.rename(columns={
    'Issue ID': 'Record ID',
    'Issue Title': 'Record Name',
    'Issue Description': 'Record Details'
}, inplace=True)

# Display the final DataFrame
print(result_df)


In [ ]:
import pandas as pd

# Read the Excel files into DataFrames
vNew = pd.read_excel('path_to_vNew.xlsx')
vOld = pd.read_excel('path_to_vOld.xlsx')

# Select relevant columns from vNew and vOld
vNew = vNew[['Employee ID', 'Position Number', 'Level4']]
vOld = vOld[['Employee ID', 'Position Number', 'Level4']]

# Merge vNew and vOld on Employee ID
merged_df = pd.merge(vNew, vOld, on='Employee ID', suffixes=('_new', '_old'), how='left')

# Rename columns to match the required format
merged_df.rename(columns={
    'Position Number_new': 'new Position Number',
    'Level4_new': 'new Level4',
    'Position Number_old': 'old Position Number',
    'Level4_old': 'old Level4'
}, inplace=True)

# Create the "Position Changed" column
merged_df['Position Changed'] = merged_df.apply(
    lambda row: 'New' if pd.isna(row['old Position Number']) or pd.isna(row['old Level4']) else 'No Change', axis=1)

# Fetch records from vOld where Employee ID is not in vNew
not_in_vNew = vOld[~vOld['Employee ID'].isin(vNew['Employee ID'])]

# Rename columns to match the format of merged_df
not_in_vNew.rename(columns={
    'Position Number': 'old Position Number',
    'Level4': 'old Level4'
}, inplace=True)

# Add new columns with default values for records not in vNew
not_in_vNew['new Position Number'] = None
not_in_vNew['new Level4'] = None
not_in_vNew['Position Changed'] = 'Left'

# Reorder columns to match the merged_df structure
not_in_vNew = not_in_vNew[['Employee ID', 'new Position Number', 'new Level4', 'old Position Number', 'old Level4', 'Position Changed']]

# Concatenate the two DataFrames
final_df = pd.concat([merged_df, not_in_vNew], ignore_index=True)

# Display the final DataFrame
print(final_df)


# Display the final DataFrame
print(merged_df)


In [ ]:
import pandas as pd

# Read the Excel files into DataFrames
vNew = pd.read_excel('path_to_vNew.xlsx')
vOld = pd.read_excel('path_to_vOld.xlsx')

# Select relevant columns from vNew and vOld
vNew = vNew[['Employee ID', 'Position Number', 'Level4']]
vOld = vOld[['Employee ID', 'Position Number', 'Level4']]

# Merge vNew and vOld on Employee ID
merged_df = pd.merge(vNew, vOld, on='Employee ID', suffixes=('_new', '_old'), how='left')

# Rename columns to match the required format
merged_df.rename(columns={'Position Number_new': 'new Position Number',
                          'Level4_new': 'new Level4',
                          'Position Number_old': 'old Position Number',
                          'Level4_old': 'old Level4'}, inplace=True)

# Display the final DataFrame
print(merged_df)


In [ ]:
import pandas as pd

# Read the Excel files into DataFrames
vNew = pd.read_excel('path_to_vNew.xlsx')
vOld = pd.read_excel('path_to_vOld.xlsx')

# Ensure column names are properly stripped of leading/trailing spaces
vNew.columns = vNew.columns.str.strip()
vOld.columns = vOld.columns.str.strip()

# Initialize new columns in vOld with default value 'left'
vOld['Pos Check'] = 'left'
vOld['BF4 Check'] = 'left'
vOld['Country Check'] = 'left'

# Perform the lookup and update the values for matching Employee Ids
vOld.loc[vOld['Employee ID'].isin(vNew['Employee ID']), 'Pos Check'] = vOld.loc[vOld['Employee ID'].isin(vNew['Employee ID']), 'Employee ID'].map(vNew.set_index('Employee ID')['Position Number'])
vOld.loc[vOld['Employee ID'].isin(vNew['Employee ID']), 'BF4 Check'] = vOld.loc[vOld['Employee ID'].isin(vNew['Employee ID']), 'Employee ID'].map(vNew.set_index('Employee ID')['BF Level 4 Name'])
vOld.loc[vOld['Employee ID'].isin(vNew['Employee ID']), 'Country Check'] = vOld.loc[vOld['Employee ID'].isin(vNew['Employee ID']), 'Employee ID'].map(vNew.set_index('Employee ID')['Work Location Country/Territory Name'])

# Create the 'In Scope' column
vOld['In Scope'] = ((vOld['Pos Check'] != vOld['Position Number']) | 
                    (vOld['BF4 Check'] != vOld['BF Level 4 Name']) | 
                    (vOld['Country Check'] != vOld['Work Location Country/Territory Name'])).apply(lambda x: 'Movement' if x else 'No Movement')

# Create 'Position Changed', 'BF Changed', and 'Country Changed' columns
vOld['Position Changed'] = vOld.apply(lambda row: 'Left' if row['Pos Check'] == 'left' else ('Yes' if row['Pos Check'] != row['Position Number'] else 'No'), axis=1)
vOld['BF Changed'] = vOld.apply(lambda row: 'Left' if row['BF4 Check'] == 'left' else ('Yes' if row['BF4 Check'] != row['BF Level 4 Name'] else 'No'), axis=1)
vOld['Country Changed'] = vOld.apply(lambda row: 'Left' if row['Country Check'] == 'left' else ('Yes' if row['Country Check'] != row['Work Location Country/Territory Name'] else 'No'), axis=1)

# Display the updated vOld DataFrame
print(vOld)


In [ ]:
import pandas as pd

# Read the Excel files into DataFrames
vNew = pd.read_excel('path_to_vNew.xlsx')
vOld = pd.read_excel('path_to_vOld.xlsx')

# Ensure column names are properly stripped of leading/trailing spaces
vNew.columns = vNew.columns.str.strip()
vOld.columns = vOld.columns.str.strip()

# Initialize new columns in vOld with default value 'left'
vOld['Pos Check'] = 'left'
vOld['BF4 Check'] = 'left'
vOld['Country Check'] = 'left'

# Perform the lookup and update the values for matching Employee Ids
vOld.loc[vOld['Employee ID'].isin(vNew['Employee ID']), 'Pos Check'] = vOld.loc[vOld['Employee ID'].isin(vNew['Employee ID']), 'Employee ID'].map(vNew.set_index('Employee ID')['Position Number'])
vOld.loc[vOld['Employee ID'].isin(vNew['Employee ID']), 'BF4 Check'] = vOld.loc[vOld['Employee ID'].isin(vNew['Employee ID']), 'Employee ID'].map(vNew.set_index('Employee ID')['BF Level 4 Name'])
vOld.loc[vOld['Employee ID'].isin(vNew['Employee ID']), 'Country Check'] = vOld.loc[vOld['Employee ID'].isin(vNew['Employee ID']), 'Employee ID'].map(vNew.set_index('Employee ID')['Work Location Country/Territory Name'])

# Display the updated vOld DataFrame
print(vOld)


In [ ]:
import pandas as pd

# Read the new and old data from the Excel workbooks
vNew = pd.read_excel('path_to_vNew.xlsx')
vOld = pd.read_excel('path_to_vOld.xlsx')

# Select the required columns
vNew = vNew[['Employee ID', 'Position Number', 'Level4']]
vOld = vOld[['Employee ID', 'Position Number', 'Level4']]

# Merge the dataframes on Employee ID
merged_df = pd.merge(vNew, vOld, on='Employee ID', how='outer', suffixes=('_new', '_old'))

# Create the Status column
merged_df['Status'] = merged_df.apply(
    lambda row: 'new' if pd.isna(row['Position Number_old']) and not pd.isna(row['Position Number_new']) else 
                ('left' if pd.isna(row['Position Number_new']) and not pd.isna(row['Position Number_old']) else 
                'existing'), axis=1)

# Save the resulting dataframe to a new Excel file
merged_df.to_excel('path_to_output.xlsx', index=False)


In [ ]:
import pandas as pd
from openpyxl import Workbook, load_workbook

# Read the Excel file
file_path = 'your_excel_file.xlsx'  # Replace with your file path
output_file_path = 'grouped_data_with_subtables.xlsx'  # Replace with your desired output file path
excel_data = pd.ExcelFile(file_path)

# Create a Pandas Excel writer object
with pd.ExcelWriter(output_file_path, engine='openpyxl') as writer:
    for sheet_name in excel_data.sheet_names:
        # Read each sheet into a DataFrame
        df = pd.read_excel(excel_data, sheet_name=sheet_name)

        # Group the data by 'Mapped L3' and 'Role_temp'
        grouped = df.groupby(['Mapped L3', 'Role_temp'])

        # Count the occurrences of 'High' and 'Low' in the 'High/Low' column
        result = grouped['High/Low'].value_counts().unstack(fill_value=0).fillna(0)

        # Add totals for each row ('Mapped L3', 'Role_temp')
        result['Total'] = result.sum(axis=1)

        # Add a totals row
        total_row = result.sum(axis=0).to_frame().T
        total_row.index = pd.MultiIndex.from_tuples([('Total', '')])
        result = pd.concat([result, total_row])

        # Reset the index to get 'Mapped L3' and 'Role_temp' as columns
        result.reset_index(inplace=True)

        # Ensure 'High' and 'Low' are present, if not add them with default 0 values
        if 'High' not in result.columns:
            result['High'] = 0
        if 'Low' not in result.columns:
            result['Low'] = 0
        result = result[['Mapped L3', 'Role_temp', 'High', 'Low', 'Total']]

        # Write the grouped data to the sheet
        result.to_excel(writer, sheet_name=sheet_name, startrow=0, index=False)
        
        # Load the workbook and access the worksheet
        workbook = writer.book
        worksheet = workbook[sheet_name]

        # Initialize start row for writing subtables
        start_row = len(result) + 2  # Move to the next row after grouped data

        # Create a list to store all subtables
        subtables_list = []

        # Create subtables for each unique 'Mapped L3'
        unique_mapped_l3 = df['Mapped L3'].unique()
        for mapped_l3 in unique_mapped_l3:
            # Skip 'Total' rows if present
            if mapped_l3 == 'Total':
                continue

            # Filter the original DataFrame based on 'Mapped L3'
            mapped_l3_data = df[df['Mapped L3'] == mapped_l3]

            # Insert a heading for unique 'Mapped L3'
            mapped_l3_heading = f"Unique L3 Value: {mapped_l3}"
            subtables_list.append((mapped_l3_heading, None))  # Append heading to list

            # Iterate over each unique 'Role_temp' for the current 'Mapped L3'
            unique_role_temp = mapped_l3_data['Role_temp'].unique()
            for role_temp in unique_role_temp:
                # Filter the data for the current 'Mapped L3' and 'Role_temp'
                subtable_data = mapped_l3_data[mapped_l3_data['Role_temp'] == role_temp]

                # Insert a heading for 'Role_temp'
                role_temp_heading = f"Role Type: {role_temp}"
                subtables_list.append((role_temp_heading, None))  # Append heading to list

                # Append the subtable data to the list
                subtables_list.append((None, subtable_data))

        # Write subtables to Excel with proper gaps
        for item in subtables_list:
            if item[0]:  # If it's a heading
                worksheet.cell(row=start_row, column=1, value=item[0])
                start_row += 2  # 2-line gap before next heading or subtable
            elif item[1] is not None:  # If it's subtable data
                item[1].to_excel(writer, sheet_name=sheet_name, startrow=start_row, index=False, header=True)
                start_row += len(item[1]) + 1  # 1-line gap after subtable

print(f"Grouped data with subtables has been saved to {output_file_path}")


In [ ]:
import pandas as pd

# Read the Excel file
file_path = 'your_excel_file.xlsx'  # Replace with your file path
output_file_path = 'grouped_data_with_subtables.xlsx'  # Replace with your desired output file path
excel_data = pd.ExcelFile(file_path)

# Create a Pandas Excel writer object
with pd.ExcelWriter(output_file_path, engine='openpyxl') as writer:
    for sheet_name in excel_data.sheet_names:
        # Read each sheet into a DataFrame
        df = pd.read_excel(excel_data, sheet_name=sheet_name)

        # Group the data by 'Mapped L3' and 'Role_temp'
        grouped = df.groupby(['Mapped L3', 'Role_temp'])

        # Count the occurrences of 'High' and 'Low' in the 'High/Low' column
        result = grouped['High/Low'].value_counts().unstack(fill_value=0).fillna(0)

        # Add totals for each row ('Mapped L3', 'Role_temp')
        result['Total'] = result.sum(axis=1)

        # Add a totals row
        total_row = result.sum(axis=0).to_frame().T
        total_row.index = pd.MultiIndex.from_tuples([('Total', '')])
        result = pd.concat([result, total_row])

        # Reset the index to get 'Mapped L3' and 'Role_temp' as columns
        result.reset_index(inplace=True)

        # Ensure 'High' and 'Low' are present, if not add them with default 0 values
        if 'High' not in result.columns:
            result['High'] = 0
        if 'Low' not in result.columns:
            result['Low'] = 0
        result = result[['Mapped L3', 'Role_temp', 'Total', 'High', 'Low']]

        # Initialize start row for writing grouped data
        start_row = 0

        # Write the grouped data to the sheet
        result.to_excel(writer, sheet_name=sheet_name, startrow=start_row, index=False)
        start_row += len(result) + 2  # Move to the next row after grouped data

        # Create subtables for each unique 'Mapped L3'
        unique_mapped_l3 = df['Mapped L3'].unique()
        for mapped_l3 in unique_mapped_l3:
            # Skip 'Total' rows if present
            if mapped_l3 == 'Total':
                continue

            # Filter the original DataFrame based on 'Mapped L3'
            mapped_l3_data = df[df['Mapped L3'] == mapped_l3]

            # Insert a heading for unique 'Mapped L3'
            mapped_l3_heading = f"Unique L3 Value: {mapped_l3}"
            worksheet = writer.sheets[sheet_name]
            worksheet.cell(row=start_row, column=1, value=mapped_l3_heading)

            # Move to the next row after writing unique L3 heading
            start_row += 2

            # Iterate over each unique 'Role_temp' for the current 'Mapped L3'
            unique_role_temp = mapped_l3_data['Role_temp'].unique()
            for role_temp in unique_role_temp:
                # Filter the data for the current 'Mapped L3' and 'Role_temp'
                subtable_data = mapped_l3_data[mapped_l3_data['Role_temp'] == role_temp]

                # Insert a heading for 'Role_temp'
                role_temp_heading = f"Role Type: {role_temp}"
                worksheet.cell(row=start_row, column=1, value=role_temp_heading)

                # Move to the next row after writing role_temp heading
                start_row += 2

                # Insert an empty row to separate headings from subtable data
                worksheet.cell(row=start_row, column=1, value="")

                # Move to the next row after inserting empty row
                start_row += 1

                # Write the subtable data to the sheet
                subtable_data.to_excel(writer, sheet_name=sheet_name, startrow=start_row, index=False)

                # Move to the next row after writing subtable data
                start_row += len(subtable_data) + 2  # Add extra space after each subtable

print(f"Grouped data with subtables has been saved to {output_file_path}")


In [12]:
import pandas as pd

# Sample data for df1 with 30 columns (only a few shown here for simplicity)
data1 = {
    'colA': [1, 2, 3, 4, 7],
    'colB': [10, 20, 30, 40, 50],
    'colS': [None, None, None, None, None]  # Initially None or some default value
    # Add other columns as needed
}
df1 = pd.DataFrame(data1)

# Sample data for df2 with 45 columns (only a few shown here for simplicity)
data2 = {
    'colR': [3, 3, 4, 5, 6, 6],
    'colS': ['a', 'b', 'c', 'd', 'e', 'f'],
    'colPM': [7, 8, 9, 10, 11, 12]
    # Add other columns as needed
}
df2 = pd.DataFrame(data2)

# Ensure the data types of colA, colR, and colPM are the same
df1['colA'] = df1['colA'].astype(str)
df2['colR'] = df2['colR'].astype(str)
df2['colPM'] = df2['colPM'].astype(str)

# Initial merge using colA and colR
df_merged_initial = pd.merge(df1, df2[['colR', 'colS']], left_on='colA', right_on='colR', how='left')

# Update df1's colS with the values from initial merge
df1['colS'] = df_merged_initial['colS_y']

# Identify rows where colS is still NaN
mask = df1['colS'].isna()

# Perform the second merge using colA and colPM for the NaN rows
df_merged_second = pd.merge(df1[mask], df2[['colPM', 'colS']], left_on='colA', right_on='colPM', how='left')

# Update colS in df1 for the NaN rows with values from the second merge
df1.loc[mask, 'colS'] = df_merged_second['colS_y']

print(df1)


  colA  colB colS
0    1    10  NaN
1    2    20  NaN
2    3    30    a
3    4    40    b
4    7    50    c


In [14]:
import pandas as pd

# Sample data for df1 with 30 columns (only a few shown here for simplicity)
data1 = {
    'colA': [1, 2, 3, 4, 7],
    'colB': [10, 20, 30, 40, 50],
    'colS': [None, '', ' ', '   ', '']  # Different types of empty or blank values
    # Add other columns as needed
}
df1 = pd.DataFrame(data1)

# Sample data for df2 with 45 columns (only a few shown here for simplicity)
data2 = {
    'colR': [3, 3, 4, 5, 6, 6],
    'colS': ['a', 'b', 'c', 'd', 'e', 'f'],
    'colPM': [7, 8, 9, 10, 11, 12]
    # Add other columns as needed
}
df2 = pd.DataFrame(data2)

# Ensure the data types of colA, colR, and colPM are the same
df1['colA'] = df1['colA'].astype(str)
df2['colR'] = df2['colR'].astype(str)
df2['colPM'] = df2['colPM'].astype(str)

# Initial merge using colA and colR
df_merged_initial = pd.merge(df1, df2[['colR', 'colS']], left_on='colA', right_on='colR', how='left')

# Update df1's colS with the values from initial merge
df1['colS'] = df_merged_initial['colS_y']

# Identify rows where colS is empty or blank
mask = df1['colS'].apply(lambda x: x == '' or x.isspace() if isinstance(x, str) else False)

# Perform the second merge using colA and colPM for the empty or blank rows
df_merged_second = pd.merge(df1[mask], df2[['colPM', 'colS']], left_on='colA', right_on='colPM', how='left')

# Update colS in df1 for the empty or blank rows with values from the second merge
df1.loc[mask, 'colS'] = df_merged_second['colS_y']

print(df1)


  colA  colB colS
0    1    10  NaN
1    2    20  NaN
2    3    30    a
3    4    40    b
4    7    50    c


In [6]:
df_merged_pm

,colA,colB,colS_x,colPM,colS_y
0,1,10,NaN,NaN,NaN
1,2,20,NaN,NaN,NaN
2,7,50,NaN,7,a


In [1]:
import pandas as pd

# Sample data for df1 with 30 columns (only a few shown here for simplicity)
data1 = {
    'colA': [1, 2, 3, 4],
    'colB': [10, 20, 30, 40],
    # Add other columns as needed
}
df1 = pd.DataFrame(data1)

# Sample data for df2 with 45 columns (only a few shown here for simplicity)
data2 = {
    'colR': [3, 3, 4, 5, 6, 6],
    'colS': ['a', 'b', 'c', 'd', 'e', 'f'],
    # Add other columns as needed
}
df2 = pd.DataFrame(data2)

# Ensure the data types of colA and colR are the same
df1['colA'] = df1['colA'].astype(str)
df2['colR'] = df2['colR'].astype(str)

# Select only the required columns from df2
df2_selected = df2[['colR', 'colS']].drop_duplicates(subset='colR')

# Merge df1 with df2_selected on colA and colR
df_merged = pd.merge(df1, df2_selected, left_on='colA', right_on='colR', how='left')

# Assign the values to the new column in df1 and drop any extra columns
df1['colS1'] = df_merged['colS']

# Drop the extra merge column if needed (not strictly necessary, but clean)
df1.drop(columns=['colR'], inplace=True, errors='ignore')

print(df1)


  colA  colB colS1
0    1    10   NaN
1    2    20   NaN
2    3    30     a
3    4    40     c


In [1]:
import pandas as pd

# Sample data for df1
data1 = {'colA': [1, 2, 3, 4]}
df1 = pd.DataFrame(data1)

# Sample data for df2 with non-unique colR values
data2 = {'colR': [3, 3, 4, 5, 6, 6], 'colS': ['a', 'b', 'c', 'd', 'e', 'f']}
df2 = pd.DataFrame(data2)

# Merge df1 with df2 on the condition that df1['colA'] matches df2['colR']
df_merged = pd.merge(df1, df2, left_on='colA', right_on='colR', how='left')

# Select only relevant columns and rename them
df1['colS1'] = df_merged['colS']

print(df1)


   colA colS1
0     1   NaN
1     2   NaN
2     3     a
3     4     b


In [2]:
import pandas as pd

# Sample DataFrames
df1 = pd.DataFrame({
    'id': [1, 2, 3, 4, 5],
    'colA': ['A1', 'A2', 'A3', 'A4', 'A5'],
    'colB': ['B1', 'B2', 'B3', 'B4', 'B5'],
    'colC': ['C1', 'C2', 'C3', 'C4', 'C5']
})

df2 = pd.DataFrame({
    'id': [6, 7, 8],
    'colX': ['X6', 'X7', 'X8'],
    'colY': ['Y6', 'Y7', 'Y8'],
    'colZ': ['Z6', 'Z7', 'Z8']
})

# Columns to copy from df1 and their corresponding columns in df2
columns_to_copy = {
    'id': 'id',
    'colB': 'colY',
    'colC': 'colZ'
}

# Create a new DataFrame with the selected columns from df1
new_rows = df1[list(columns_to_copy.keys())].copy()

# Rename the columns in the new DataFrame to match the column names in df2
new_rows.rename(columns=columns_to_copy, inplace=True)

# Append the new DataFrame to df2
df2 = df2.append(new_rows, ignore_index=True)

# Display the updated df2
print(df2)


   id colX colY colZ
0   6   X6   Y6   Z6
1   7   X7   Y7   Z7
2   8   X8   Y8   Z8
3   1  NaN   B1   C1
4   2  NaN   B2   C2
5   3  NaN   B3   C3
6   4  NaN   B4   C4
7   5  NaN   B5   C5
